# 🛡️ Phase 4: Adversarial Robustness Evaluation

## Tri-Objective Robust XAI for Medical Imaging

**Author:** Viraj Pankaj Jain  
**Institution:** University of Glasgow  
**Date:** November 2025

---

## 📋 Overview

This notebook evaluates the **adversarial robustness** of baseline ResNet-50 models trained on ISIC 2018 dermoscopy images. We systematically assess vulnerability to:

| Attack | Type | Strength | Use Case |
|--------|------|----------|----------|
| **FGSM** | Gradient-based | Fast, single-step | Real-time threat assessment |
| **PGD** | Iterative | Strong, multi-step | Reliable robustness benchmark |
| **C&W** | Optimization | Strongest, minimal perturbation | Worst-case security analysis |

## 🎯 Research Questions Addressed

- **RQ1:** How vulnerable are standard CNNs to adversarial attacks in medical imaging?
- **RQ2:** How does attack strength (ε) affect model accuracy degradation?
- **RQ3:** Which skin lesion classes are most vulnerable to adversarial perturbations?

## 📊 Evaluation Protocol

```
┌─────────────────────────────────────────────────────────────────┐
│  ADVERSARIAL EVALUATION PIPELINE                                │
├─────────────────────────────────────────────────────────────────┤
│  1. Load trained baseline models (Seeds: 42, 123, 456)          │
│  2. Evaluate clean accuracy (sanity check)                      │
│  3. Generate adversarial examples at ε ∈ {2/255, 4/255, 8/255}  │
│  4. Measure robust accuracy under each attack                   │
│  5. Analyze per-class vulnerability                             │
│  6. Visualize perturbations and decision boundaries             │
│  7. Statistical analysis across seeds                           │
└─────────────────────────────────────────────────────────────────┘
```

## ⚡ Hardware Requirements

- **Recommended:** NVIDIA A100 (40GB) - Full evaluation ~15 minutes
- **Minimum:** NVIDIA T4 (16GB) - Full evaluation ~45 minutes

---

In [ ]:
# ============================================================================
# CELL 1: ENVIRONMENT SETUP
# ============================================================================
import sys
import os
from pathlib import Path

print("=" * 70)
print("PHASE 4: ADVERSARIAL ROBUSTNESS EVALUATION")
print("=" * 70)

# Mount Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
    print("✅ Google Colab detected, Drive mounted")
except ImportError:
    IN_COLAB = False
    print("✅ Local environment detected")

# Clone/update repository
if IN_COLAB:
    REPO_PATH = Path('/content/tri-objective-robust-xai-medimg')
    if not REPO_PATH.exists():
        !git clone https://github.com/viraj1011JAIN/tri-objective-robust-xai-medimg.git {REPO_PATH}
        print("✅ Repository cloned")
    else:
        os.chdir(REPO_PATH)
        !git pull origin main
        print("✅ Repository updated")

    os.chdir(REPO_PATH)
    sys.path.insert(0, str(REPO_PATH))
    PROJECT_ROOT = REPO_PATH
else:
    PROJECT_ROOT = Path.cwd().parent
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"📁 Project root: {PROJECT_ROOT}")

In [ ]:
# ============================================================================
# CELL 2: INSTALL DEPENDENCIES
# ============================================================================
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install -q timm albumentations scikit-learn pandas matplotlib seaborn tqdm mlflow
!pip install -q plotly kaleido scipy statsmodels
print("✅ Dependencies installed")

In [ ]:
# ============================================================================
# CELL 3: IMPORTS
# ============================================================================
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm.auto import tqdm
from pathlib import Path
import json
import time
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any
import warnings
warnings.filterwarnings('ignore')

# Albumentations for transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

# Metrics
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score,
    confusion_matrix, roc_auc_score
)
from scipy import stats

# Project imports - Attacks
from src.attacks.fgsm import FGSM, FGSMConfig
from src.attacks.pgd import PGD, PGDConfig
from src.attacks.cw import CarliniWagner, CWConfig

# Project imports - Data & Model
from src.datasets.isic import ISICDataset
from src.models.build import build_model
from src.utils.reproducibility import set_global_seed

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    # Enable TF32 for A100
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cudnn.benchmark = True

In [ ]:
# ============================================================================
# CELL 4: CONFIGURATION
# ============================================================================
print("=" * 70)
print("CONFIGURATION")
print("=" * 70)

CONFIG = {
    # Data paths (Google Drive)
    'data_root': Path('/content/drive/MyDrive/data/data/isic_2018'),
    'checkpoint_dir': Path('/content/drive/MyDrive/checkpoints/baseline'),
    'results_dir': Path('/content/drive/MyDrive/results/phase4'),

    # Model
    'model_name': 'resnet50',
    'num_classes': 7,

    # Evaluation settings
    'batch_size': 64,
    'num_workers': 4,
    'image_size': 224,

    # Seeds to evaluate
    'seeds': [42, 123, 456],

    # Attack configurations
    'epsilons': [2/255, 4/255, 8/255],
    'pgd_steps': 40,
    'cw_iterations': 100,

    # Class names
    'class_names': ['AKIEC', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'VASC'],
}

# ImageNet normalization
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

# Create output directories
CONFIG['results_dir'].mkdir(parents=True, exist_ok=True)
(CONFIG['results_dir'] / 'figures').mkdir(exist_ok=True)

print(f"📊 Model: {CONFIG['model_name']}")
print(f"📊 Seeds: {CONFIG['seeds']}")
print(f"📊 Epsilons: {[f'{int(e*255)}/255' for e in CONFIG['epsilons']]}")
print(f"📊 Batch size: {CONFIG['batch_size']}")
print(f"📁 Data: {CONFIG['data_root']}")
print(f"📁 Checkpoints: {CONFIG['checkpoint_dir']}")
print(f"📁 Results: {CONFIG['results_dir']}")

In [ ]:
# ============================================================================
# CELL 5: DATA PREPARATION
# ============================================================================
print("=" * 70)
print("DATA PREPARATION")
print("=" * 70)

# Load and fix metadata
metadata_path = CONFIG['data_root'] / 'metadata.csv'
print(f"📄 Loading metadata: {metadata_path}")

df = pd.read_csv(metadata_path)
print(f"   Total samples: {len(df)}")

# Fix path separators
if 'image_path' in df.columns:
    df['image_path'] = df['image_path'].str.replace('\\', '/', regex=False)
    print("   ✅ Fixed path separators")

# Save fixed metadata
fixed_path = CONFIG['data_root'] / 'metadata_fixed.csv'
df.to_csv(fixed_path, index=False)

# Show test split info
test_df = df[df['split'] == 'test']
print(f"\n📊 Test samples: {len(test_df)}")
print(f"📊 Test class distribution:")
print(test_df['label'].value_counts())

In [ ]:
# ============================================================================
# CELL 6: CREATE TEST DATASET
# ============================================================================
print("=" * 70)
print("CREATING TEST DATASET")
print("=" * 70)

# Test transforms - NO normalization (attacks need [0,1] range)
test_transforms = A.Compose([
    A.Resize(CONFIG['image_size'], CONFIG['image_size']),
    A.Normalize(mean=[0, 0, 0], std=[1, 1, 1]),  # Keep in [0, 1]
    ToTensorV2()
])

# Create test dataset
test_dataset = ISICDataset(
    root=str(CONFIG['data_root']),
    split='test',
    transforms=test_transforms,
    csv_path=str(fixed_path),
    image_column='image_path',
    label_column='label'
)

# Create dataloader
test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG['batch_size'],
    shuffle=False,
    num_workers=CONFIG['num_workers'],
    pin_memory=True
)

print(f"✅ Test samples: {len(test_dataset)}")
print(f"✅ Batches: {len(test_loader)}")
print(f"✅ Classes: {CONFIG['class_names']}")

In [ ]:
# ============================================================================
# CELL 7: HELPER FUNCTIONS
# ============================================================================

def get_normalizer(device):
    """Create ImageNet normalization function."""
    mean = torch.tensor(IMAGENET_MEAN).view(1, 3, 1, 1).to(device)
    std = torch.tensor(IMAGENET_STD).view(1, 3, 1, 1).to(device)
    
    def normalize(x):
        return (x - mean) / std
    return normalize

def evaluate_clean(model, dataloader, device, normalize_fn):
    """Evaluate model on clean data."""
    model.eval()
    all_preds, all_labels, all_probs = [], [], []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc='Clean Eval', leave=False):
            if len(batch) == 3:
                images, labels, _ = batch
            else:
                images, labels = batch
            
            images = images.to(device)
            labels = labels.to(device)
            
            # Normalize and predict
            outputs = model(normalize_fn(images))
            probs = F.softmax(outputs, dim=1)
            preds = outputs.argmax(dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    all_probs = np.array(all_probs)
    
    return {
        'accuracy': accuracy_score(all_labels, all_preds) * 100,
        'balanced_accuracy': balanced_accuracy_score(all_labels, all_preds) * 100,
        'f1_macro': f1_score(all_labels, all_preds, average='macro') * 100,
        'auroc': roc_auc_score(all_labels, all_probs, multi_class='ovr') * 100,
        'predictions': all_preds,
        'labels': all_labels,
        'probs': all_probs
    }

def evaluate_attack(model, dataloader, device, normalize_fn, attack_fn, desc='Attack'):
    """Evaluate model under adversarial attack."""
    model.eval()
    all_clean_preds, all_adv_preds, all_labels = [], [], []
    
    for batch in tqdm(dataloader, desc=desc, leave=False):
        if len(batch) == 3:
            images, labels, _ = batch
        else:
            images, labels = batch
        
        images = images.to(device)
        labels = labels.to(device)
        
        # Clean predictions
        with torch.no_grad():
            clean_preds = model(normalize_fn(images)).argmax(dim=1)
        
        # Generate adversarial examples
        x_adv = attack_fn(images, labels)
        
        # Adversarial predictions
        with torch.no_grad():
            adv_preds = model(normalize_fn(x_adv)).argmax(dim=1)
        
        all_clean_preds.extend(clean_preds.cpu().numpy())
        all_adv_preds.extend(adv_preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    
    all_clean_preds = np.array(all_clean_preds)
    all_adv_preds = np.array(all_adv_preds)
    all_labels = np.array(all_labels)
    
    clean_acc = accuracy_score(all_labels, all_clean_preds) * 100
    robust_acc = accuracy_score(all_labels, all_adv_preds) * 100
    
    # Per-class robust accuracy
    cm = confusion_matrix(all_labels, all_adv_preds)
    per_class_acc = (cm.diagonal() / cm.sum(axis=1)) * 100
    
    return {
        'clean_accuracy': clean_acc,
        'robust_accuracy': robust_acc,
        'accuracy_drop': clean_acc - robust_acc,
        'per_class_robust_acc': dict(zip(CONFIG['class_names'], per_class_acc)),
        'confusion_matrix': cm
    }

print("✅ Helper functions defined")

In [ ]:
# ============================================================================
# CELL 8: LOAD MODELS AND VERIFY CLEAN ACCURACY
# ============================================================================
print("=" * 70)
print("LOADING MODELS & VERIFYING CLEAN ACCURACY")
print("=" * 70)

normalize = get_normalizer(device)
models = {}
clean_results = {}

for seed in CONFIG['seeds']:
    print(f"\n📥 Loading seed {seed}...")
    
    # Find checkpoint
    checkpoint_path = CONFIG['checkpoint_dir'] / f'seed_{seed}' / 'best.pt'
    if not checkpoint_path.exists():
        print(f"   ❌ Checkpoint not found: {checkpoint_path}")
        continue
    
    # Load model
    model = build_model(
        architecture=CONFIG['model_name'],
        num_classes=CONFIG['num_classes'],
        pretrained=False
    ).to(device)
    
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    models[seed] = model
    
    # Verify clean accuracy
    result = evaluate_clean(model, test_loader, device, normalize)
    clean_results[seed] = result
    
    print(f"   ✅ Clean Accuracy: {result['accuracy']:.2f}%")
    print(f"   ✅ AUROC: {result['auroc']:.2f}%")

# Summary
print(f"\n{'='*70}")
print(f"📊 CLEAN ACCURACY SUMMARY")
print(f"{'='*70}")
accs = [clean_results[s]['accuracy'] for s in clean_results]
print(f"Mean: {np.mean(accs):.2f}% ± {np.std(accs):.2f}%")
for seed in clean_results:
    print(f"   Seed {seed}: {clean_results[seed]['accuracy']:.2f}%")

In [ ]:
# ============================================================================
# CELL 9: RUN ADVERSARIAL EVALUATION - ALL ATTACKS
# ============================================================================
print("=" * 70)
print("ADVERSARIAL EVALUATION - ALL ATTACKS")
print("=" * 70)
print(f"⏱️  Start time: {datetime.now().strftime('%H:%M:%S')}")

all_results = {}

for seed in CONFIG['seeds']:
    if seed not in models:
        continue
    
    print(f"\n{'='*70}")
    print(f"SEED {seed}")
    print(f"{'='*70}")
    
    model = models[seed]
    seed_results = {'clean': clean_results[seed]}
    
    # ==================== FGSM ====================
    print("\n🔥 FGSM Attacks:")
    for eps in CONFIG['epsilons']:
        eps_str = f"{int(eps*255)}/255"
        
        # Create FGSM attack
        fgsm_config = FGSMConfig(
            epsilon=eps,
            clip_min=0.0,
            clip_max=1.0,
            targeted=False
        )
        fgsm = FGSM(fgsm_config)
        
        def fgsm_attack_fn(x, y):
            return fgsm.generate(model, x, y, loss_fn=nn.CrossEntropyLoss(), normalize=normalize)
        
        result = evaluate_attack(model, test_loader, device, normalize, fgsm_attack_fn, f"FGSM ε={eps_str}")
        seed_results[f'fgsm_{eps_str}'] = result
        print(f"   ε={eps_str}: {result['robust_accuracy']:.2f}% (drop: {result['accuracy_drop']:.2f}%)")
    
    # ==================== PGD ====================
    print("\n🔥 PGD Attacks:")
    for eps in CONFIG['epsilons']:
        eps_str = f"{int(eps*255)}/255"
        step_size = eps / 4
        
        # Create PGD attack
        pgd_config = PGDConfig(
            epsilon=eps,
            num_steps=CONFIG['pgd_steps'],
            step_size=step_size,
            random_start=True,
            clip_min=0.0,
            clip_max=1.0,
            targeted=False
        )
        pgd = PGD(pgd_config)
        
        def pgd_attack_fn(x, y):
            return pgd.generate(model, x, y, loss_fn=nn.CrossEntropyLoss(), normalize=normalize)
        
        result = evaluate_attack(model, test_loader, device, normalize, pgd_attack_fn, f"PGD ε={eps_str}")
        seed_results[f'pgd_{eps_str}'] = result
        print(f"   ε={eps_str}: {result['robust_accuracy']:.2f}% (drop: {result['accuracy_drop']:.2f}%)")
    
    # ==================== C&W ====================
    print("\n🔥 Carlini-Wagner Attack:")
    cw_config = CWConfig(
        confidence=0.0,
        learning_rate=0.01,
        max_iterations=CONFIG['cw_iterations'],
        binary_search_steps=5,
        clip_min=0.0,
        clip_max=1.0,
        targeted=False
    )
    cw = CarliniWagner(cw_config)
    
    def cw_attack_fn(x, y):
        return cw.generate(model, x, y, normalize=normalize)
    
    result = evaluate_attack(model, test_loader, device, normalize, cw_attack_fn, "C&W L2")
    seed_results['cw'] = result
    print(f"   C&W: {result['robust_accuracy']:.2f}% (drop: {result['accuracy_drop']:.2f}%)")
    
    all_results[seed] = seed_results

print(f"\n⏱️  End time: {datetime.now().strftime('%H:%M:%S')}")
print("✅ Evaluation complete!")

In [ ]:
# ============================================================================
# CELL 10: AGGREGATE RESULTS TABLE
# ============================================================================
print("=" * 70)
print("AGGREGATED RESULTS")
print("=" * 70)

# Build results table
rows = []
attacks = ['clean'] + [f'fgsm_{int(e*255)}/255' for e in CONFIG['epsilons']] + \
          [f'pgd_{int(e*255)}/255' for e in CONFIG['epsilons']] + ['cw']

for attack in attacks:
    if attack == 'clean':
        accs = [all_results[s]['clean']['accuracy'] for s in all_results]
        attack_name = 'Clean (No Attack)'
    elif attack == 'cw':
        accs = [all_results[s]['cw']['robust_accuracy'] for s in all_results]
        attack_name = 'C&W L2'
    else:
        accs = [all_results[s][attack]['robust_accuracy'] for s in all_results]
        attack_name = attack.upper().replace('_', ' ε=')
    
    rows.append({
        'Attack': attack_name,
        'Mean Accuracy (%)': f"{np.mean(accs):.2f}",
        'Std (%)': f"{np.std(accs):.2f}",
        'Seed 42': f"{accs[0]:.2f}" if len(accs) > 0 else 'N/A',
        'Seed 123': f"{accs[1]:.2f}" if len(accs) > 1 else 'N/A',
        'Seed 456': f"{accs[2]:.2f}" if len(accs) > 2 else 'N/A',
    })

results_df = pd.DataFrame(rows)
print(results_df.to_string(index=False))

# Save to CSV
results_df.to_csv(CONFIG['results_dir'] / 'adversarial_results.csv', index=False)
print(f"\n✅ Results saved to: {CONFIG['results_dir'] / 'adversarial_results.csv'}")

In [ ]:
# ============================================================================
# CELL 11: VISUALIZATION - ROBUSTNESS DEGRADATION CURVES
# ============================================================================
print("=" * 70)
print("VISUALIZATIONS")
print("=" * 70)

# Prepare data for plotting
eps_vals = [0] + [e*255 for e in CONFIG['epsilons']]  # 0, 2, 4, 8

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

colors = ['#2ecc71', '#3498db', '#e74c3c']

# Plot FGSM
ax = axes[0]
for i, seed in enumerate(CONFIG['seeds']):
    if seed not in all_results:
        continue
    accs = [all_results[seed]['clean']['accuracy']]
    for eps in CONFIG['epsilons']:
        accs.append(all_results[seed][f'fgsm_{int(eps*255)}/255']['robust_accuracy'])
    ax.plot(eps_vals, accs, 'o-', color=colors[i], label=f'Seed {seed}', linewidth=2, markersize=8)

ax.set_xlabel('Perturbation ε (×255)', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('FGSM Attack Robustness', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xticks(eps_vals)
ax.set_ylim(0, 100)

# Plot PGD
ax = axes[1]
for i, seed in enumerate(CONFIG['seeds']):
    if seed not in all_results:
        continue
    accs = [all_results[seed]['clean']['accuracy']]
    for eps in CONFIG['epsilons']:
        accs.append(all_results[seed][f'pgd_{int(eps*255)}/255']['robust_accuracy'])
    ax.plot(eps_vals, accs, 'o-', color=colors[i], label=f'Seed {seed}', linewidth=2, markersize=8)

ax.set_xlabel('Perturbation ε (×255)', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('PGD Attack Robustness (40 steps)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xticks(eps_vals)
ax.set_ylim(0, 100)

plt.suptitle('Adversarial Robustness: Baseline ResNet-50 on ISIC 2018', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(CONFIG['results_dir'] / 'figures' / 'robustness_curves.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved: robustness_curves.png")

In [ ]:
# ============================================================================
# CELL 12: PER-CLASS VULNERABILITY HEATMAP
# ============================================================================

# Aggregate per-class accuracy across seeds for PGD ε=8/255
class_accs = {}
for cls in CONFIG['class_names']:
    class_accs[cls] = []
    for seed in all_results:
        class_accs[cls].append(all_results[seed]['pgd_8/255']['per_class_robust_acc'][cls])

# Create heatmap data
heatmap_data = []
for attack_key in ['fgsm_8/255', 'pgd_8/255', 'cw']:
    row = []
    for cls in CONFIG['class_names']:
        accs = [all_results[s][attack_key]['per_class_robust_acc'][cls] for s in all_results]
        row.append(np.mean(accs))
    heatmap_data.append(row)

fig, ax = plt.subplots(figsize=(12, 4))
im = ax.imshow(heatmap_data, cmap='RdYlGn', aspect='auto', vmin=0, vmax=100)

ax.set_xticks(range(len(CONFIG['class_names'])))
ax.set_xticklabels(CONFIG['class_names'], fontsize=11)
ax.set_yticks(range(3))
ax.set_yticklabels(['FGSM ε=8/255', 'PGD ε=8/255', 'C&W L2'], fontsize=11)

# Add text annotations
for i in range(3):
    for j in range(len(CONFIG['class_names'])):
        text = ax.text(j, i, f"{heatmap_data[i][j]:.1f}%",
                       ha='center', va='center', color='black', fontsize=10, fontweight='bold')

ax.set_title('Per-Class Robust Accuracy Under Strong Attacks\n(Mean across 3 seeds)', fontsize=14, fontweight='bold')
plt.colorbar(im, ax=ax, label='Accuracy (%)')
plt.tight_layout()
plt.savefig(CONFIG['results_dir'] / 'figures' / 'class_vulnerability_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved: class_vulnerability_heatmap.png")

In [ ]:
# ============================================================================
# CELL 13: ATTACK COMPARISON BAR CHART
# ============================================================================

# Prepare data
attack_names = ['Clean', 'FGSM\nε=2/255', 'FGSM\nε=4/255', 'FGSM\nε=8/255',
                'PGD\nε=2/255', 'PGD\nε=4/255', 'PGD\nε=8/255', 'C&W']
attack_keys = ['clean', 'fgsm_2/255', 'fgsm_4/255', 'fgsm_8/255',
               'pgd_2/255', 'pgd_4/255', 'pgd_8/255', 'cw']

# Calculate mean and std
means = []
stds = []
for key in attack_keys:
    if key == 'clean':
        accs = [all_results[s]['clean']['accuracy'] for s in all_results]
    else:
        accs = [all_results[s][key]['robust_accuracy'] for s in all_results]
    means.append(np.mean(accs))
    stds.append(np.std(accs))

# Create bar chart
fig, ax = plt.subplots(figsize=(12, 6))

colors = ['#2ecc71'] + ['#3498db']*3 + ['#e74c3c']*3 + ['#9b59b6']
bars = ax.bar(attack_names, means, yerr=stds, capsize=5, color=colors, edgecolor='black', alpha=0.8)

# Add value labels
for bar, mean in zip(bars, means):
    ax.annotate(f'{mean:.1f}%', xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                xytext=(0, 5), textcoords='offset points', ha='center', fontsize=10, fontweight='bold')

ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('Baseline Model Robustness Comparison\n(Mean ± Std across 3 seeds)', fontsize=14, fontweight='bold')
ax.set_ylim(0, 100)
ax.axhline(y=means[0], color='gray', linestyle='--', alpha=0.5, label='Clean Accuracy')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(CONFIG['results_dir'] / 'figures' / 'attack_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Saved: attack_comparison.png")

In [ ]:
# ============================================================================
# CELL 14: SAVE ALL RESULTS
# ============================================================================
print("=" * 70)
print("SAVING RESULTS")
print("=" * 70)

# Prepare JSON-serializable results
export_results = {}
for seed in all_results:
    export_results[str(seed)] = {}
    for attack_key, attack_result in all_results[seed].items():
        export_results[str(seed)][attack_key] = {
            k: v.tolist() if isinstance(v, np.ndarray) else v
            for k, v in attack_result.items()
            if k not in ['predictions', 'labels', 'probs', 'confusion_matrix']
        }

# Save to JSON
results_file = CONFIG['results_dir'] / 'adversarial_results.json'
with open(results_file, 'w') as f:
    json.dump(export_results, f, indent=2)
print(f"✅ Results saved to: {results_file}")

# List all saved files
print(f"\n📁 Saved files in {CONFIG['results_dir']}:")
for f in sorted(CONFIG['results_dir'].glob('**/*')):
    if f.is_file():
        size_kb = f.stat().st_size / 1024
        print(f"   📄 {f.name} ({size_kb:.1f} KB)")

In [ ]:
# ============================================================================
# CELL 15: FINAL SUMMARY
# ============================================================================
print("=" * 70)
print("PHASE 4 COMPLETE - FINAL SUMMARY")
print("=" * 70)

# Calculate key metrics
clean_mean = np.mean([all_results[s]['clean']['accuracy'] for s in all_results])
fgsm8_mean = np.mean([all_results[s]['fgsm_8/255']['robust_accuracy'] for s in all_results])
pgd8_mean = np.mean([all_results[s]['pgd_8/255']['robust_accuracy'] for s in all_results])
cw_mean = np.mean([all_results[s]['cw']['robust_accuracy'] for s in all_results])

print(f"\n📊 BASELINE MODEL ROBUSTNESS (Mean across seeds):")
print(f"-" * 50)
print(f"   Clean Accuracy:       {clean_mean:.2f}%")
print(f"   FGSM (ε=8/255):       {fgsm8_mean:.2f}%  (drop: {clean_mean - fgsm8_mean:.2f}%)")
print(f"   PGD-40 (ε=8/255):     {pgd8_mean:.2f}%  (drop: {clean_mean - pgd8_mean:.2f}%)")
print(f"   C&W L2:               {cw_mean:.2f}%  (drop: {clean_mean - cw_mean:.2f}%)")

print(f"\n🔑 KEY FINDINGS:")
print(f"-" * 50)
print(f"   • Standard CNNs are highly vulnerable to adversarial attacks")
print(f"   • PGD causes {clean_mean - pgd8_mean:.1f}% accuracy drop at ε=8/255")
print(f"   • Adversarial training (Phase 5) is needed for robustness")

print(f"\n🎯 NEXT STEPS:")
print(f"-" * 50)
print(f"   1. Run Phase 5: Tri-objective robust training")
print(f"   2. Run Phase 6: Explainability analysis (Grad-CAM, SHAP)")
print(f"   3. Compare baseline vs robust models")

print(f"\n" + "=" * 70)
print("✅ PHASE 4 ADVERSARIAL EVALUATION COMPLETE!")
print("=" * 70)

In [ ]:
#@title 🔧 Cell 2: Environment Setup & Dependencies
#@markdown **Run this cell first to install all required packages**

import subprocess
import sys

def install_packages():
    """Install required packages for adversarial evaluation."""
    packages = [
        "torch>=2.0.0",
        "torchvision>=0.15.0",
        "timm>=0.9.0",
        "albumentations>=1.3.0",
        "scikit-learn>=1.3.0",
        "pandas>=2.0.0",
        "numpy>=1.24.0",
        "matplotlib>=3.7.0",
        "seaborn>=0.12.0",
        "plotly>=5.15.0",
        "kaleido",  # For plotly static export
        "tqdm>=4.65.0",
        "mlflow>=2.5.0",
        "scipy>=1.11.0",
    ]
    
    print("📦 Installing packages...")
    for pkg in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])
    print("✅ All packages installed!")

# Check if running in Colab
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Running in Google Colab")
    install_packages()
except ImportError:
    IN_COLAB = False
    print("💻 Running locally")

# Core imports
import os
import gc
import json
import time
import warnings
from pathlib import Path
from datetime import datetime
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple, Any

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ML utilities
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score
)
from tqdm.auto import tqdm

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Set visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams.update({
    'figure.figsize': (12, 8),
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.dpi': 100,
})

# GPU Configuration
print("\n" + "="*60)
print("🖥️  HARDWARE CONFIGURATION")
print("="*60)

if torch.cuda.is_available():
    device = torch.device("cuda")
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU: {gpu_name}")
    print(f"✅ VRAM: {gpu_mem:.1f} GB")
    
    # Enable optimizations for A100/Ampere GPUs
    if "A100" in gpu_name or torch.cuda.get_device_capability()[0] >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True
        print("✅ TF32 enabled for Ampere GPU")
    
    torch.backends.cudnn.benchmark = True
    print("✅ cuDNN benchmark enabled")
else:
    device = torch.device("cpu")
    print("⚠️ No GPU found, using CPU (will be slow)")

print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ Device: {device}")
print("="*60)

In [ ]:
#@title 🗂️ Cell 3: Mount Google Drive & Configure Paths
#@markdown **Configure data and checkpoint paths**

# Mount Google Drive (Colab only)
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    print("✅ Google Drive mounted")

# ============================================================================
# PATH CONFIGURATION - Adjust these paths as needed
# ============================================================================

@dataclass
class PathConfig:
    """Central path configuration for the evaluation pipeline."""
    
    # Base paths
    drive_base: Path = Path("/content/drive/MyDrive")
    
    # Data paths
    data_root: Path = field(default=None)
    train_dir: Path = field(default=None)
    val_dir: Path = field(default=None)
    test_dir: Path = field(default=None)
    
    # Checkpoint paths
    checkpoint_dir: Path = field(default=None)
    
    # Output paths
    results_dir: Path = field(default=None)
    figures_dir: Path = field(default=None)
    
    def __post_init__(self):
        """Initialize derived paths."""
        if self.data_root is None:
            self.data_root = self.drive_base / "data" / "data" / "isic_2018"
        if self.train_dir is None:
            self.train_dir = self.data_root / "train"
        if self.val_dir is None:
            self.val_dir = self.data_root / "val"
        if self.test_dir is None:
            self.test_dir = self.data_root / "test"
        if self.checkpoint_dir is None:
            self.checkpoint_dir = self.drive_base / "checkpoints" / "baseline"
        if self.results_dir is None:
            self.results_dir = self.drive_base / "results" / "phase4_adversarial"
        if self.figures_dir is None:
            self.figures_dir = self.results_dir / "figures"
    
    def validate(self) -> bool:
        """Validate that required paths exist."""
        required = [self.data_root, self.test_dir, self.checkpoint_dir]
        missing = [p for p in required if not p.exists()]
        
        if missing:
            print("❌ Missing paths:")
            for p in missing:
                print(f"   - {p}")
            return False
        return True
    
    def create_output_dirs(self):
        """Create output directories if they don't exist."""
        self.results_dir.mkdir(parents=True, exist_ok=True)
        self.figures_dir.mkdir(parents=True, exist_ok=True)
        print(f"✅ Results directory: {self.results_dir}")
        print(f"✅ Figures directory: {self.figures_dir}")

# Initialize paths
paths = PathConfig()

# Validate paths
print("\n" + "="*60)
print("📁 PATH VALIDATION")
print("="*60)

if paths.validate():
    print(f"✅ Data root: {paths.data_root}")
    print(f"✅ Test directory: {paths.test_dir}")
    print(f"✅ Checkpoint directory: {paths.checkpoint_dir}")
    paths.create_output_dirs()
else:
    print("\n⚠️ Please update PathConfig with correct paths!")
    
# List available checkpoints
print("\n📦 Available Checkpoints:")
if paths.checkpoint_dir.exists():
    checkpoints = list(paths.checkpoint_dir.glob("*.pt"))
    for ckpt in sorted(checkpoints):
        size_mb = ckpt.stat().st_size / 1e6
        print(f"   - {ckpt.name} ({size_mb:.1f} MB)")
else:
    print("   ⚠️ No checkpoints found!")

print("="*60)

In [ ]:
#@title 📥 Cell 4: Clone Repository & Import Attack Classes
#@markdown **Clone the project repository and import custom modules**

import os
import sys

# Clone repository (Colab only)
REPO_URL = "https://github.com/viraj1011JAIN/tri-objective-robust-xai-medimg.git"
REPO_DIR = "/content/tri-objective-robust-xai-medimg"

if IN_COLAB:
    if not os.path.exists(REPO_DIR):
        print(f"📥 Cloning repository...")
        os.system(f"git clone {REPO_URL} {REPO_DIR}")
        print("✅ Repository cloned!")
    else:
        print("📁 Repository already exists, pulling latest...")
        os.system(f"cd {REPO_DIR} && git pull")
    
    # Add to Python path
    if REPO_DIR not in sys.path:
        sys.path.insert(0, REPO_DIR)
    print(f"✅ Added {REPO_DIR} to Python path")
else:
    # Local development - find project root
    current_dir = Path.cwd()
    if "notebooks" in str(current_dir):
        project_root = current_dir.parent
    else:
        project_root = current_dir
    
    if str(project_root) not in sys.path:
        sys.path.insert(0, str(project_root))
    print(f"✅ Using local project: {project_root}")

# Import custom attack classes
print("\n🔧 Importing attack modules...")

try:
    from src.attacks.fgsm import FGSM, FGSMConfig, fgsm_attack
    from src.attacks.pgd import PGD, PGDConfig, pgd_attack
    from src.attacks.cw import CarliniWagner, CWConfig, cw_attack
    from src.attacks.base import AttackConfig, AttackResult
    print("✅ FGSM attack imported")
    print("✅ PGD attack imported")
    print("✅ Carlini-Wagner attack imported")
    print("✅ Base attack classes imported")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("⚠️ Please ensure the repository is properly cloned")
    raise

# Import dataset utilities
try:
    from src.datasets.isic import ISICDataset
    print("✅ ISICDataset imported")
except ImportError:
    print("⚠️ ISICDataset not found, will use custom implementation")
    ISICDataset = None

# Import model utilities
import timm
print(f"✅ timm version: {timm.__version__}")

print("\n" + "="*60)
print("✅ ALL MODULES IMPORTED SUCCESSFULLY")
print("="*60)

In [ ]:
#@title 📊 Cell 5: Dataset & Model Loading Utilities
#@markdown **Define dataset wrapper and model loading functions**

import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
from torch.utils.data import Dataset, DataLoader

# ============================================================================
# ISIC 2018 CLASS INFORMATION
# ============================================================================

CLASS_NAMES = [
    "AKIEC",  # Actinic Keratoses
    "BCC",    # Basal Cell Carcinoma
    "BKL",    # Benign Keratosis
    "DF",     # Dermatofibroma
    "MEL",    # Melanoma
    "NV",     # Melanocytic Nevi
    "VASC"    # Vascular Lesions
]

CLASS_DESCRIPTIONS = {
    "AKIEC": "Actinic Keratoses (pre-cancerous)",
    "BCC": "Basal Cell Carcinoma (cancerous)",
    "BKL": "Benign Keratosis (non-cancerous)",
    "DF": "Dermatofibroma (benign)",
    "MEL": "Melanoma (malignant, dangerous)",
    "NV": "Melanocytic Nevi (common moles)",
    "VASC": "Vascular Lesions (blood vessel)"
}

NUM_CLASSES = len(CLASS_NAMES)

# ImageNet normalization (used by pretrained models)
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

# ============================================================================
# DATASET CLASS
# ============================================================================

class ISICTestDataset(Dataset):
    """
    ISIC 2018 Test Dataset for adversarial evaluation.
    
    Returns unnormalized images in [0, 1] range for adversarial attacks.
    Normalization is applied separately during model inference.
    """
    
    def __init__(
        self,
        root_dir: Path,
        transform: Optional[A.Compose] = None,
        max_samples: Optional[int] = None
    ):
        self.root_dir = Path(root_dir)
        self.transform = transform or self._default_transform()
        
        # Collect samples
        self.samples = []
        self.class_to_idx = {name: idx for idx, name in enumerate(CLASS_NAMES)}
        
        for class_name in CLASS_NAMES:
            class_dir = self.root_dir / class_name
            if class_dir.exists():
                images = list(class_dir.glob("*.jpg")) + list(class_dir.glob("*.png"))
                for img_path in images:
                    self.samples.append((img_path, self.class_to_idx[class_name]))
        
        # Limit samples if specified
        if max_samples and len(self.samples) > max_samples:
            # Stratified sampling
            from collections import defaultdict
            by_class = defaultdict(list)
            for path, label in self.samples:
                by_class[label].append((path, label))
            
            per_class = max_samples // NUM_CLASSES
            self.samples = []
            for label, items in by_class.items():
                self.samples.extend(items[:per_class])
        
        print(f"📊 Loaded {len(self.samples)} test samples from {self.root_dir}")
        
        # Print class distribution
        class_counts = {}
        for _, label in self.samples:
            class_counts[label] = class_counts.get(label, 0) + 1
        for idx, name in enumerate(CLASS_NAMES):
            print(f"   {name}: {class_counts.get(idx, 0)} samples")
    
    def _default_transform(self) -> A.Compose:
        """Default test transform: resize and convert to tensor."""
        return A.Compose([
            A.Resize(224, 224),
            A.Normalize(mean=[0, 0, 0], std=[1, 1, 1]),  # Keep in [0, 1]
            ToTensorV2()
        ])
    
    def __len__(self) -> int:
        return len(self.samples)
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, int, dict]:
        img_path, label = self.samples[idx]
        
        # Load image
        image = np.array(Image.open(img_path).convert("RGB"))
        
        # Apply transforms
        if self.transform:
            transformed = self.transform(image=image)
            image = transformed["image"]
        
        # Metadata
        metadata = {
            "image_path": str(img_path),
            "class_name": CLASS_NAMES[label]
        }
        
        return image.float(), label, metadata

# ============================================================================
# MODEL LOADING
# ============================================================================

def create_model(num_classes: int = NUM_CLASSES, pretrained: bool = False) -> nn.Module:
    """Create ResNet-50 model for ISIC classification."""
    model = timm.create_model(
        "resnet50",
        pretrained=pretrained,
        num_classes=num_classes
    )
    return model

def load_checkpoint(model: nn.Module, checkpoint_path: Path, device: torch.device) -> dict:
    """
    Load model checkpoint and return metadata.
    
    Args:
        model: PyTorch model
        checkpoint_path: Path to checkpoint file
        device: Target device
    
    Returns:
        Checkpoint metadata dictionary
    """
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
    
    # Handle different checkpoint formats
    if "model_state_dict" in checkpoint:
        model.load_state_dict(checkpoint["model_state_dict"])
        metadata = {
            "epoch": checkpoint.get("epoch", "unknown"),
            "val_acc": checkpoint.get("val_acc", checkpoint.get("best_val_acc", "unknown")),
            "seed": checkpoint.get("seed", "unknown")
        }
    elif "state_dict" in checkpoint:
        model.load_state_dict(checkpoint["state_dict"])
        metadata = {"epoch": "unknown", "val_acc": "unknown", "seed": "unknown"}
    else:
        # Direct state dict
        model.load_state_dict(checkpoint)
        metadata = {"epoch": "unknown", "val_acc": "unknown", "seed": "unknown"}
    
    model.to(device)
    model.eval()
    
    return metadata

def get_normalizer(device: torch.device):
    """Get ImageNet normalization function."""
    mean = torch.tensor(IMAGENET_MEAN).view(1, 3, 1, 1).to(device)
    std = torch.tensor(IMAGENET_STD).view(1, 3, 1, 1).to(device)
    
    def normalize(x: torch.Tensor) -> torch.Tensor:
        return (x - mean) / std
    
    return normalize

print("✅ Dataset and model utilities defined")

In [ ]:
#@title ⚙️ Cell 6: Evaluation Configuration
#@markdown **Configure attack parameters and evaluation settings**

@dataclass
class EvaluationConfig:
    """Configuration for adversarial evaluation."""
    
    # Seeds to evaluate
    seeds: List[int] = field(default_factory=lambda: [42, 123, 456])
    
    # Epsilon values (perturbation budgets)
    epsilons: List[float] = field(default_factory=lambda: [2/255, 4/255, 8/255])
    
    # Attack configurations
    fgsm_enabled: bool = True
    pgd_enabled: bool = True
    pgd_steps: int = 40
    pgd_step_size: Optional[float] = None  # Auto: epsilon/4
    
    cw_enabled: bool = True
    cw_iterations: int = 100  # Reduced for speed (default 1000)
    cw_confidence: float = 0.0
    cw_learning_rate: float = 0.01
    
    # Evaluation settings
    batch_size: int = 64  # Increase for A100
    num_workers: int = 4
    max_test_samples: Optional[int] = None  # None = all samples
    
    # Output settings
    save_adversarial_examples: bool = True
    num_examples_to_save: int = 50
    
    def __post_init__(self):
        """Adjust settings based on hardware."""
        if torch.cuda.is_available():
            gpu_name = torch.cuda.get_device_name(0)
            gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
            
            # Optimize batch size for GPU memory
            if gpu_mem >= 35:  # A100 40GB
                self.batch_size = 128
                self.num_workers = 8
                self.cw_iterations = 200
                print(f"⚡ A100 optimizations: batch={self.batch_size}, C&W iters={self.cw_iterations}")
            elif gpu_mem >= 14:  # T4/V100
                self.batch_size = 64
                self.num_workers = 4
                self.cw_iterations = 100
                print(f"⚡ T4/V100 settings: batch={self.batch_size}")
            else:  # Smaller GPU
                self.batch_size = 32
                self.num_workers = 2
                self.cw_iterations = 50
                print(f"⚠️ Limited GPU: batch={self.batch_size}")
    
    def get_epsilon_str(self, eps: float) -> str:
        """Convert epsilon to readable string."""
        return f"{int(eps * 255)}/255"
    
    def summary(self) -> str:
        """Get configuration summary."""
        lines = [
            "="*60,
            "📋 EVALUATION CONFIGURATION",
            "="*60,
            f"Seeds: {self.seeds}",
            f"Epsilons: {[self.get_epsilon_str(e) for e in self.epsilons]}",
            "",
            "Attacks:",
            f"  FGSM: {'✓' if self.fgsm_enabled else '✗'}",
            f"  PGD:  {'✓' if self.pgd_enabled else '✗'} (steps={self.pgd_steps})",
            f"  C&W:  {'✓' if self.cw_enabled else '✗'} (iters={self.cw_iterations})",
            "",
            f"Batch size: {self.batch_size}",
            f"Max samples: {self.max_test_samples or 'all'}",
            "="*60
        ]
        return "\n".join(lines)

# Initialize configuration
config = EvaluationConfig()
print(config.summary())

# Epsilon display helper
EPSILON_LABELS = {
    2/255: "ε=2/255 (weak)",
    4/255: "ε=4/255 (medium)",
    8/255: "ε=8/255 (strong)"
}

print("\n✅ Configuration ready")

In [ ]:
#@title 🎯 Cell 7: Adversarial Attack Engine
#@markdown **Core attack generation and evaluation functions**

class AdversarialEvaluator:
    """
    Unified adversarial evaluation engine.
    
    Supports FGSM, PGD, and Carlini-Wagner attacks with batch processing
    and detailed metrics collection.
    """
    
    def __init__(
        self,
        model: nn.Module,
        device: torch.device,
        normalize_fn: callable
    ):
        self.model = model
        self.device = device
        self.normalize = normalize_fn
        self.model.eval()
        
        # Results storage
        self.results = {}
    
    def evaluate_clean(
        self,
        dataloader: DataLoader,
        desc: str = "Clean Evaluation"
    ) -> Dict[str, Any]:
        """Evaluate model on clean (unperturbed) data."""
        all_preds = []
        all_labels = []
        all_probs = []
        
        with torch.no_grad():
            for batch in tqdm(dataloader, desc=desc, leave=False):
                # Handle 3-tuple (image, label, metadata)
                images, labels = batch[0], batch[1]
                images = images.to(self.device)
                labels = labels.to(self.device)
                
                # Forward pass with normalization
                logits = self.model(self.normalize(images))
                probs = F.softmax(logits, dim=1)
                preds = logits.argmax(dim=1)
                
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_probs.extend(probs.cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        all_probs = np.array(all_probs)
        
        # Calculate metrics
        accuracy = accuracy_score(all_labels, all_preds)
        balanced_acc = balanced_accuracy_score(all_labels, all_preds)
        f1_macro = f1_score(all_labels, all_preds, average='macro')
        f1_weighted = f1_score(all_labels, all_preds, average='weighted')
        
        # Per-class accuracy
        cm = confusion_matrix(all_labels, all_preds)
        per_class_acc = cm.diagonal() / cm.sum(axis=1)
        
        # AUROC (one-vs-rest)
        try:
            auroc = roc_auc_score(all_labels, all_probs, multi_class='ovr')
        except ValueError:
            auroc = 0.0
        
        return {
            "accuracy": accuracy,
            "balanced_accuracy": balanced_acc,
            "f1_macro": f1_macro,
            "f1_weighted": f1_weighted,
            "auroc": auroc,
            "per_class_accuracy": dict(zip(CLASS_NAMES, per_class_acc)),
            "confusion_matrix": cm,
            "predictions": all_preds,
            "labels": all_labels,
            "probabilities": all_probs
        }
    
    def evaluate_attack(
        self,
        dataloader: DataLoader,
        attack_name: str,
        epsilon: float,
        attack_fn: callable,
        desc: str = None
    ) -> Dict[str, Any]:
        """
        Evaluate model under adversarial attack.
        
        Args:
            dataloader: Test data loader
            attack_name: Name of attack (FGSM, PGD, CW)
            epsilon: Perturbation budget
            attack_fn: Function that generates adversarial examples
            desc: Progress bar description
        
        Returns:
            Dictionary with attack results and metrics
        """
        if desc is None:
            desc = f"{attack_name} ε={config.get_epsilon_str(epsilon)}"
        
        all_preds_clean = []
        all_preds_adv = []
        all_labels = []
        all_l2_dists = []
        all_linf_dists = []
        successful_attacks = 0
        total_samples = 0
        
        # Store some examples for visualization
        saved_examples = []
        
        for batch in tqdm(dataloader, desc=desc, leave=False):
            images, labels = batch[0], batch[1]
            images = images.to(self.device)
            labels = labels.to(self.device)
            
            # Clean predictions
            with torch.no_grad():
                clean_logits = self.model(self.normalize(images))
                clean_preds = clean_logits.argmax(dim=1)
            
            # Generate adversarial examples
            try:
                x_adv = attack_fn(images, labels)
            except Exception as e:
                print(f"⚠️ Attack failed on batch: {e}")
                continue
            
            # Adversarial predictions
            with torch.no_grad():
                adv_logits = self.model(self.normalize(x_adv))
                adv_preds = adv_logits.argmax(dim=1)
            
            # Calculate perturbation norms
            delta = (x_adv - images).view(images.size(0), -1)
            l2_dist = torch.norm(delta, p=2, dim=1)
            linf_dist = torch.norm(delta, p=float('inf'), dim=1)
            
            # Track successful attacks (correctly classified → misclassified)
            was_correct = (clean_preds == labels)
            is_wrong = (adv_preds != labels)
            successful = was_correct & is_wrong
            
            successful_attacks += successful.sum().item()
            total_samples += was_correct.sum().item()  # Only count correctly classified
            
            # Store results
            all_preds_clean.extend(clean_preds.cpu().numpy())
            all_preds_adv.extend(adv_preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_l2_dists.extend(l2_dist.cpu().numpy())
            all_linf_dists.extend(linf_dist.cpu().numpy())
            
            # Save examples for visualization
            if len(saved_examples) < config.num_examples_to_save:
                for i in range(min(5, images.size(0))):
                    if len(saved_examples) >= config.num_examples_to_save:
                        break
                    saved_examples.append({
                        "clean": images[i].cpu(),
                        "adversarial": x_adv[i].cpu(),
                        "perturbation": (x_adv[i] - images[i]).cpu(),
                        "true_label": labels[i].item(),
                        "clean_pred": clean_preds[i].item(),
                        "adv_pred": adv_preds[i].item()
                    })
        
        # Convert to arrays
        all_preds_clean = np.array(all_preds_clean)
        all_preds_adv = np.array(all_preds_adv)
        all_labels = np.array(all_labels)
        
        # Calculate metrics
        clean_acc = accuracy_score(all_labels, all_preds_clean)
        robust_acc = accuracy_score(all_labels, all_preds_adv)
        attack_success_rate = successful_attacks / max(total_samples, 1)
        
        # Per-class robust accuracy
        cm_adv = confusion_matrix(all_labels, all_preds_adv)
        per_class_robust_acc = cm_adv.diagonal() / cm_adv.sum(axis=1)
        
        return {
            "attack_name": attack_name,
            "epsilon": epsilon,
            "epsilon_str": config.get_epsilon_str(epsilon),
            "clean_accuracy": clean_acc,
            "robust_accuracy": robust_acc,
            "accuracy_drop": clean_acc - robust_acc,
            "attack_success_rate": attack_success_rate,
            "mean_l2_dist": np.mean(all_l2_dists),
            "mean_linf_dist": np.mean(all_linf_dists),
            "per_class_robust_accuracy": dict(zip(CLASS_NAMES, per_class_robust_acc)),
            "confusion_matrix": cm_adv,
            "saved_examples": saved_examples,
            "predictions_clean": all_preds_clean,
            "predictions_adv": all_preds_adv,
            "labels": all_labels
        }
    
    def create_fgsm_attack(self, epsilon: float) -> callable:
        """Create FGSM attack function."""
        fgsm_config = FGSMConfig(
            epsilon=epsilon,
            clip_min=0.0,
            clip_max=1.0,
            targeted=False,
            device=str(self.device)
        )
        attack = FGSM(fgsm_config)
        
        def attack_fn(x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
            return attack.generate(
                self.model, x, y,
                loss_fn=nn.CrossEntropyLoss(),
                normalize=self.normalize
            )
        return attack_fn
    
    def create_pgd_attack(self, epsilon: float, num_steps: int = 40) -> callable:
        """Create PGD attack function."""
        step_size = epsilon / 4  # Standard choice
        
        pgd_config = PGDConfig(
            epsilon=epsilon,
            num_steps=num_steps,
            step_size=step_size,
            random_start=True,
            early_stop=False,
            clip_min=0.0,
            clip_max=1.0,
            targeted=False,
            device=str(self.device)
        )
        attack = PGD(pgd_config)
        
        def attack_fn(x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
            return attack.generate(
                self.model, x, y,
                loss_fn=nn.CrossEntropyLoss(),
                normalize=self.normalize
            )
        return attack_fn
    
    def create_cw_attack(
        self,
        confidence: float = 0.0,
        max_iterations: int = 100,
        learning_rate: float = 0.01
    ) -> callable:
        """Create Carlini-Wagner L2 attack function."""
        cw_config = CWConfig(
            confidence=confidence,
            learning_rate=learning_rate,
            max_iterations=max_iterations,
            binary_search_steps=5,  # Reduced for speed
            initial_c=1e-3,
            clip_min=0.0,
            clip_max=1.0,
            targeted=False,
            device=str(self.device)
        )
        attack = CarliniWagner(cw_config)
        
        def attack_fn(x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
            return attack.generate(
                self.model, x, y,
                normalize=self.normalize
            )
        return attack_fn

print("✅ AdversarialEvaluator class defined")

In [ ]:
#@title 📦 Cell 8: Load Test Dataset and Models
#@markdown **Load test data and all seed checkpoints**

print("="*60)
print("📊 LOADING TEST DATASET")
print("="*60)

# Create test dataset
test_dataset = ISICTestDataset(
    root_dir=paths.test_dir,
    max_samples=config.max_test_samples
)

# Create dataloader
test_loader = DataLoader(
    test_dataset,
    batch_size=config.batch_size,
    shuffle=False,
    num_workers=config.num_workers,
    pin_memory=True
)

print(f"\n✅ Test samples: {len(test_dataset)}")
print(f"✅ Batches: {len(test_loader)}")
print(f"✅ Batch size: {config.batch_size}")

# ============================================================================
# LOAD MODELS FOR ALL SEEDS
# ============================================================================

print("\n" + "="*60)
print("🔧 LOADING MODEL CHECKPOINTS")
print("="*60)

models = {}
normalize = get_normalizer(device)

for seed in config.seeds:
    checkpoint_path = paths.checkpoint_dir / f"baseline_seed_{seed}.pt"
    
    if not checkpoint_path.exists():
        # Try alternative naming
        alt_path = paths.checkpoint_dir / f"seed_{seed}_best.pt"
        if alt_path.exists():
            checkpoint_path = alt_path
        else:
            print(f"⚠️ Checkpoint not found for seed {seed}")
            continue
    
    print(f"\n📥 Loading seed {seed}...")
    model = create_model(num_classes=NUM_CLASSES, pretrained=False)
    metadata = load_checkpoint(model, checkpoint_path, device)
    
    models[seed] = model
    
    print(f"   ✅ Loaded: {checkpoint_path.name}")
    print(f"   📈 Validation accuracy: {metadata.get('val_acc', 'N/A')}")

print(f"\n✅ Loaded {len(models)} models")

# ============================================================================
# SANITY CHECK: VERIFY CLEAN ACCURACY
# ============================================================================

print("\n" + "="*60)
print("🔍 SANITY CHECK: CLEAN ACCURACY")
print("="*60)

clean_results = {}

for seed, model in models.items():
    print(f"\n🧪 Evaluating seed {seed}...")
    evaluator = AdversarialEvaluator(model, device, normalize)
    result = evaluator.evaluate_clean(test_loader, desc=f"Clean eval (seed {seed})")
    clean_results[seed] = result
    
    print(f"   ✅ Accuracy: {result['accuracy']*100:.2f}%")
    print(f"   ✅ Balanced Accuracy: {result['balanced_accuracy']*100:.2f}%")
    print(f"   ✅ F1 (macro): {result['f1_macro']*100:.2f}%")
    print(f"   ✅ AUROC: {result['auroc']*100:.2f}%")

# Summary statistics
mean_acc = np.mean([r['accuracy'] for r in clean_results.values()])
std_acc = np.std([r['accuracy'] for r in clean_results.values()])

print(f"\n📊 Mean Clean Accuracy: {mean_acc*100:.2f}% ± {std_acc*100:.2f}%")
print("="*60)

In [ ]:
#@title 🚀 Cell 9: Run Full Adversarial Evaluation
#@markdown **Execute FGSM, PGD, and C&W attacks across all seeds and epsilons**

print("="*70)
print("🚀 STARTING FULL ADVERSARIAL EVALUATION")
print("="*70)
print(f"⏱️  Start time: {datetime.now().strftime('%H:%M:%S')}")
print(f"🎯 Seeds: {config.seeds}")
print(f"🎯 Epsilons: {[config.get_epsilon_str(e) for e in config.epsilons]}")
print(f"🎯 Attacks: FGSM={'✓' if config.fgsm_enabled else '✗'}, "
      f"PGD={'✓' if config.pgd_enabled else '✗'}, "
      f"C&W={'✓' if config.cw_enabled else '✗'}")
print("="*70)

# Results storage
all_results = {
    "clean": clean_results,
    "fgsm": {},
    "pgd": {},
    "cw": {}
}

start_time = time.time()

for seed in config.seeds:
    if seed not in models:
        continue
    
    model = models[seed]
    evaluator = AdversarialEvaluator(model, device, normalize)
    
    print(f"\n{'='*60}")
    print(f"🔬 EVALUATING SEED {seed}")
    print(f"{'='*60}")
    
    # Initialize seed results
    all_results["fgsm"][seed] = {}
    all_results["pgd"][seed] = {}
    all_results["cw"][seed] = {}
    
    # ========================================================================
    # FGSM ATTACKS
    # ========================================================================
    if config.fgsm_enabled:
        print(f"\n⚡ FGSM Attacks (Seed {seed})")
        print("-" * 40)
        
        for eps in config.epsilons:
            attack_fn = evaluator.create_fgsm_attack(eps)
            result = evaluator.evaluate_attack(
                test_loader,
                attack_name="FGSM",
                epsilon=eps,
                attack_fn=attack_fn
            )
            all_results["fgsm"][seed][eps] = result
            
            print(f"   ε={config.get_epsilon_str(eps):>7}: "
                  f"Robust Acc = {result['robust_accuracy']*100:5.2f}% "
                  f"(↓{result['accuracy_drop']*100:5.2f}pp)")
    
    # ========================================================================
    # PGD ATTACKS
    # ========================================================================
    if config.pgd_enabled:
        print(f"\n🔄 PGD-{config.pgd_steps} Attacks (Seed {seed})")
        print("-" * 40)
        
        for eps in config.epsilons:
            attack_fn = evaluator.create_pgd_attack(eps, num_steps=config.pgd_steps)
            result = evaluator.evaluate_attack(
                test_loader,
                attack_name=f"PGD-{config.pgd_steps}",
                epsilon=eps,
                attack_fn=attack_fn
            )
            all_results["pgd"][seed][eps] = result
            
            print(f"   ε={config.get_epsilon_str(eps):>7}: "
                  f"Robust Acc = {result['robust_accuracy']*100:5.2f}% "
                  f"(↓{result['accuracy_drop']*100:5.2f}pp)")
    
    # ========================================================================
    # C&W ATTACKS (Only at strongest epsilon for speed)
    # ========================================================================
    if config.cw_enabled:
        print(f"\n🎯 C&W L2 Attack (Seed {seed})")
        print("-" * 40)
        
        # C&W is epsilon-free (L2 minimization), run once
        attack_fn = evaluator.create_cw_attack(
            confidence=config.cw_confidence,
            max_iterations=config.cw_iterations,
            learning_rate=config.cw_learning_rate
        )
        result = evaluator.evaluate_attack(
            test_loader,
            attack_name="C&W-L2",
            epsilon=0.0,  # C&W minimizes L2 directly
            attack_fn=attack_fn,
            desc="C&W L2 Attack"
        )
        all_results["cw"][seed]["l2"] = result
        
        print(f"   Robust Acc = {result['robust_accuracy']*100:5.2f}% "
              f"(↓{result['accuracy_drop']*100:5.2f}pp)")
        print(f"   Mean L2 perturbation: {result['mean_l2_dist']:.4f}")
    
    # Memory cleanup
    gc.collect()
    torch.cuda.empty_cache()

# Timing
elapsed = time.time() - start_time
print(f"\n{'='*70}")
print(f"✅ EVALUATION COMPLETE")
print(f"⏱️  Total time: {elapsed/60:.1f} minutes")
print(f"{'='*70}")

In [ ]:
#@title 📊 Cell 10: Results Summary Table
#@markdown **Generate comprehensive results summary with statistics**

print("="*70)
print("📊 ADVERSARIAL ROBUSTNESS RESULTS SUMMARY")
print("="*70)

# ============================================================================
# BUILD RESULTS DATAFRAME
# ============================================================================

results_data = []

for seed in config.seeds:
    if seed not in clean_results:
        continue
    
    # Clean accuracy
    clean_acc = clean_results[seed]["accuracy"]
    
    # FGSM results
    if config.fgsm_enabled and seed in all_results["fgsm"]:
        for eps in config.epsilons:
            if eps in all_results["fgsm"][seed]:
                r = all_results["fgsm"][seed][eps]
                results_data.append({
                    "Seed": seed,
                    "Attack": "FGSM",
                    "Epsilon": config.get_epsilon_str(eps),
                    "Epsilon_Val": eps,
                    "Clean_Acc": clean_acc * 100,
                    "Robust_Acc": r["robust_accuracy"] * 100,
                    "Acc_Drop": r["accuracy_drop"] * 100,
                    "Attack_Success": r["attack_success_rate"] * 100,
                    "Mean_Linf": r["mean_linf_dist"]
                })
    
    # PGD results
    if config.pgd_enabled and seed in all_results["pgd"]:
        for eps in config.epsilons:
            if eps in all_results["pgd"][seed]:
                r = all_results["pgd"][seed][eps]
                results_data.append({
                    "Seed": seed,
                    "Attack": f"PGD-{config.pgd_steps}",
                    "Epsilon": config.get_epsilon_str(eps),
                    "Epsilon_Val": eps,
                    "Clean_Acc": clean_acc * 100,
                    "Robust_Acc": r["robust_accuracy"] * 100,
                    "Acc_Drop": r["accuracy_drop"] * 100,
                    "Attack_Success": r["attack_success_rate"] * 100,
                    "Mean_Linf": r["mean_linf_dist"]
                })
    
    # C&W results
    if config.cw_enabled and seed in all_results["cw"]:
        if "l2" in all_results["cw"][seed]:
            r = all_results["cw"][seed]["l2"]
            results_data.append({
                "Seed": seed,
                "Attack": "C&W-L2",
                "Epsilon": "N/A",
                "Epsilon_Val": 0,
                "Clean_Acc": clean_acc * 100,
                "Robust_Acc": r["robust_accuracy"] * 100,
                "Acc_Drop": r["accuracy_drop"] * 100,
                "Attack_Success": r["attack_success_rate"] * 100,
                "Mean_L2": r["mean_l2_dist"]
            })

df_results = pd.DataFrame(results_data)

# ============================================================================
# AGGREGATE STATISTICS (Mean ± Std across seeds)
# ============================================================================

print("\n📈 AGGREGATED RESULTS (Mean ± Std across seeds)")
print("-" * 70)

summary_data = []

# Group by Attack and Epsilon
for attack in df_results["Attack"].unique():
    for eps in df_results[df_results["Attack"] == attack]["Epsilon"].unique():
        subset = df_results[(df_results["Attack"] == attack) & (df_results["Epsilon"] == eps)]
        
        if len(subset) > 0:
            summary_data.append({
                "Attack": attack,
                "Epsilon": eps,
                "Clean_Acc": f"{subset['Clean_Acc'].mean():.2f} ± {subset['Clean_Acc'].std():.2f}",
                "Robust_Acc": f"{subset['Robust_Acc'].mean():.2f} ± {subset['Robust_Acc'].std():.2f}",
                "Acc_Drop": f"{subset['Acc_Drop'].mean():.2f} ± {subset['Acc_Drop'].std():.2f}",
                "Attack_Success": f"{subset['Attack_Success'].mean():.2f} ± {subset['Attack_Success'].std():.2f}"
            })

df_summary = pd.DataFrame(summary_data)
print(df_summary.to_string(index=False))

# ============================================================================
# DETAILED PER-SEED TABLE
# ============================================================================

print("\n\n📋 DETAILED RESULTS (Per Seed)")
print("-" * 70)

display_cols = ["Seed", "Attack", "Epsilon", "Clean_Acc", "Robust_Acc", "Acc_Drop", "Attack_Success"]
print(df_results[display_cols].to_string(index=False))

# ============================================================================
# KEY FINDINGS
# ============================================================================

print("\n\n🔑 KEY FINDINGS")
print("=" * 70)

# Best/worst robust accuracy under PGD-8/255
if config.pgd_enabled:
    pgd_8 = df_results[(df_results["Attack"] == f"PGD-{config.pgd_steps}") & 
                       (df_results["Epsilon"] == "8/255")]
    if len(pgd_8) > 0:
        mean_robust = pgd_8["Robust_Acc"].mean()
        mean_drop = pgd_8["Acc_Drop"].mean()
        print(f"• PGD-{config.pgd_steps} (ε=8/255): {mean_robust:.2f}% robust accuracy "
              f"(↓{mean_drop:.2f}pp from clean)")

# FGSM vs PGD comparison
if config.fgsm_enabled and config.pgd_enabled:
    fgsm_8 = df_results[(df_results["Attack"] == "FGSM") & (df_results["Epsilon"] == "8/255")]
    pgd_8 = df_results[(df_results["Attack"] == f"PGD-{config.pgd_steps}") & 
                       (df_results["Epsilon"] == "8/255")]
    if len(fgsm_8) > 0 and len(pgd_8) > 0:
        fgsm_robust = fgsm_8["Robust_Acc"].mean()
        pgd_robust = pgd_8["Robust_Acc"].mean()
        print(f"• FGSM vs PGD gap at ε=8/255: {abs(fgsm_robust - pgd_robust):.2f}pp")

# C&W results
if config.cw_enabled:
    cw_results = df_results[df_results["Attack"] == "C&W-L2"]
    if len(cw_results) > 0:
        mean_cw_robust = cw_results["Robust_Acc"].mean()
        print(f"• C&W L2 attack: {mean_cw_robust:.2f}% robust accuracy")

print("=" * 70)

In [ ]:
#@title 📈 Cell 11: PhD-Level Visualization - Robustness Degradation Curves
#@markdown **Publication-quality robustness vs perturbation strength plots**

def create_robustness_curves(df_results: pd.DataFrame, config: EvaluationConfig) -> go.Figure:
    """
    Create interactive robustness degradation curves.
    
    Shows how accuracy degrades with increasing perturbation budget ε.
    """
    # Prepare data for plotting
    attacks = ["FGSM", f"PGD-{config.pgd_steps}"]
    colors = {"FGSM": "#FF6B6B", f"PGD-{config.pgd_steps}": "#4ECDC4"}
    markers = {"FGSM": "circle", f"PGD-{config.pgd_steps}": "square"}
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=(
            "<b>Robustness Degradation by Attack Type</b>",
            "<b>Accuracy Drop Severity</b>"
        ),
        horizontal_spacing=0.12
    )
    
    # Add clean accuracy reference line
    clean_acc = df_results["Clean_Acc"].mean()
    
    for attack in attacks:
        attack_data = df_results[df_results["Attack"] == attack].copy()
        if len(attack_data) == 0:
            continue
        
        # Group by epsilon
        grouped = attack_data.groupby("Epsilon_Val").agg({
            "Robust_Acc": ["mean", "std"],
            "Acc_Drop": ["mean", "std"]
        }).reset_index()
        grouped.columns = ["Epsilon", "Robust_Mean", "Robust_Std", "Drop_Mean", "Drop_Std"]
        grouped = grouped.sort_values("Epsilon")
        
        # Convert epsilon to string labels for x-axis
        epsilon_labels = [f"{int(e*255)}/255" for e in grouped["Epsilon"]]
        
        # Plot 1: Robustness curves with confidence bands
        fig.add_trace(
            go.Scatter(
                x=epsilon_labels,
                y=grouped["Robust_Mean"],
                mode="lines+markers",
                name=attack,
                line=dict(color=colors[attack], width=3),
                marker=dict(size=12, symbol=markers[attack]),
                error_y=dict(
                    type="data",
                    array=grouped["Robust_Std"],
                    visible=True,
                    color=colors[attack],
                    thickness=2
                ),
                legendgroup=attack,
                showlegend=True
            ),
            row=1, col=1
        )
        
        # Plot 2: Accuracy drop bars
        fig.add_trace(
            go.Bar(
                x=epsilon_labels,
                y=grouped["Drop_Mean"],
                name=attack,
                marker_color=colors[attack],
                error_y=dict(
                    type="data",
                    array=grouped["Drop_Std"],
                    visible=True
                ),
                legendgroup=attack,
                showlegend=False
            ),
            row=1, col=2
        )
    
    # Add clean accuracy reference
    fig.add_hline(
        y=clean_acc, 
        line_dash="dash", 
        line_color="gray",
        annotation_text=f"Clean: {clean_acc:.1f}%",
        row=1, col=1
    )
    
    # Update layout
    fig.update_layout(
        height=500,
        width=1100,
        title=dict(
            text="<b>Adversarial Robustness Analysis: Baseline ResNet-50 on ISIC 2018</b>",
            font=dict(size=18),
            x=0.5
        ),
        font=dict(family="Arial", size=12),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.08,
            xanchor="center",
            x=0.5
        ),
        paper_bgcolor="white",
        plot_bgcolor="white"
    )
    
    # Axis labels
    fig.update_xaxes(title_text="Perturbation Budget (ε)", row=1, col=1, gridcolor="lightgray")
    fig.update_xaxes(title_text="Perturbation Budget (ε)", row=1, col=2, gridcolor="lightgray")
    fig.update_yaxes(title_text="Robust Accuracy (%)", row=1, col=1, gridcolor="lightgray", range=[0, 100])
    fig.update_yaxes(title_text="Accuracy Drop (pp)", row=1, col=2, gridcolor="lightgray")
    
    return fig

# Create and display
fig_robustness = create_robustness_curves(df_results, config)
fig_robustness.show()

# Save figure
if paths.figures_dir.exists():
    fig_robustness.write_html(paths.figures_dir / "robustness_curves.html")
    fig_robustness.write_image(paths.figures_dir / "robustness_curves.png", scale=2)
    print(f"✅ Saved to {paths.figures_dir / 'robustness_curves.png'}")

In [ ]:
#@title 🔥 Cell 12: PhD-Level Visualization - Per-Class Vulnerability Heatmap
#@markdown **Detailed heatmap showing which skin lesion classes are most vulnerable**

def create_vulnerability_heatmap(all_results: dict, clean_results: dict, config: EvaluationConfig) -> go.Figure:
    """
    Create per-class vulnerability heatmap across attacks and epsilons.
    """
    # Build vulnerability matrix
    attack_configs = []
    
    # Add FGSM configs
    if config.fgsm_enabled:
        for eps in config.epsilons:
            attack_configs.append(("FGSM", eps, f"FGSM ε={config.get_epsilon_str(eps)}"))
    
    # Add PGD configs
    if config.pgd_enabled:
        for eps in config.epsilons:
            attack_configs.append((f"PGD-{config.pgd_steps}", eps, f"PGD-{config.pgd_steps} ε={config.get_epsilon_str(eps)}"))
    
    # Add C&W
    if config.cw_enabled:
        attack_configs.append(("C&W-L2", "l2", "C&W L2"))
    
    # Calculate mean per-class robust accuracy across seeds
    vulnerability_matrix = []
    
    for class_name in CLASS_NAMES:
        row = []
        for attack_type, eps_key, label in attack_configs:
            # Get attack dict key
            attack_key = "fgsm" if attack_type == "FGSM" else ("pgd" if "PGD" in attack_type else "cw")
            
            accuracies = []
            for seed in config.seeds:
                if seed not in all_results[attack_key]:
                    continue
                if eps_key not in all_results[attack_key][seed]:
                    continue
                
                result = all_results[attack_key][seed][eps_key]
                if class_name in result.get("per_class_robust_accuracy", {}):
                    accuracies.append(result["per_class_robust_accuracy"][class_name] * 100)
            
            if accuracies:
                row.append(np.mean(accuracies))
            else:
                row.append(np.nan)
        
        vulnerability_matrix.append(row)
    
    vulnerability_matrix = np.array(vulnerability_matrix)
    
    # Create heatmap
    fig = go.Figure(data=go.Heatmap(
        z=vulnerability_matrix,
        x=[c[2] for c in attack_configs],
        y=[f"{name}\n({CLASS_DESCRIPTIONS[name].split(' ')[0]})" for name in CLASS_NAMES],
        colorscale=[
            [0, "#d73027"],      # Red - low accuracy (vulnerable)
            [0.25, "#fc8d59"],   # Orange
            [0.5, "#fee08b"],    # Yellow
            [0.75, "#91cf60"],   # Light green
            [1, "#1a9850"]       # Dark green - high accuracy (robust)
        ],
        colorbar=dict(
            title="Robust<br>Accuracy (%)",
            titleside="right",
            ticksuffix="%"
        ),
        text=np.round(vulnerability_matrix, 1),
        texttemplate="%{text:.1f}%",
        textfont=dict(size=11, color="black"),
        hoverongaps=False,
        hovertemplate="<b>%{y}</b><br>Attack: %{x}<br>Robust Acc: %{z:.2f}%<extra></extra>"
    ))
    
    # Add clean accuracy comparison as annotation column
    fig.update_layout(
        title=dict(
            text="<b>Per-Class Adversarial Vulnerability Analysis</b><br>"
                 "<sup>Red = Vulnerable (low accuracy) | Green = Robust (high accuracy)</sup>",
            font=dict(size=16),
            x=0.5
        ),
        xaxis=dict(
            title="Attack Configuration",
            tickangle=45,
            side="bottom"
        ),
        yaxis=dict(
            title="Skin Lesion Class",
            autorange="reversed"
        ),
        height=550,
        width=1000,
        font=dict(family="Arial", size=12),
        paper_bgcolor="white",
        plot_bgcolor="white"
    )
    
    return fig

# Create and display
fig_heatmap = create_vulnerability_heatmap(all_results, clean_results, config)
fig_heatmap.show()

# Save
if paths.figures_dir.exists():
    fig_heatmap.write_html(paths.figures_dir / "vulnerability_heatmap.html")
    fig_heatmap.write_image(paths.figures_dir / "vulnerability_heatmap.png", scale=2)
    print(f"✅ Saved to {paths.figures_dir / 'vulnerability_heatmap.png'}")

# ============================================================================
# IDENTIFY MOST VULNERABLE CLASSES
# ============================================================================

print("\n🎯 CLASS VULNERABILITY RANKING (Under PGD-40 ε=8/255)")
print("-" * 60)

# Get PGD-40 at 8/255 per-class results
eps_8 = 8/255
class_vuln = {}

for seed in config.seeds:
    if seed not in all_results["pgd"]:
        continue
    if eps_8 not in all_results["pgd"][seed]:
        continue
    
    result = all_results["pgd"][seed][eps_8]
    for class_name, acc in result.get("per_class_robust_accuracy", {}).items():
        if class_name not in class_vuln:
            class_vuln[class_name] = []
        class_vuln[class_name].append(acc * 100)

# Sort by vulnerability (lowest accuracy = most vulnerable)
sorted_vuln = sorted(
    [(name, np.mean(accs), np.std(accs)) for name, accs in class_vuln.items()],
    key=lambda x: x[1]
)

for rank, (name, mean_acc, std_acc) in enumerate(sorted_vuln, 1):
    status = "🔴 CRITICAL" if mean_acc < 20 else "🟡 MODERATE" if mean_acc < 50 else "🟢 ROBUST"
    print(f"{rank}. {name:6} ({CLASS_DESCRIPTIONS[name]:30}): {mean_acc:5.1f}% ± {std_acc:4.1f}% {status}")

In [ ]:
#@title 🎯 Cell 13: PhD-Level Visualization - Clean vs Adversarial Confusion Matrices
#@markdown **Side-by-side confusion matrices showing attack impact on predictions**

def create_confusion_matrix_comparison(
    clean_results: dict, 
    all_results: dict, 
    seed: int,
    attack_key: str = "pgd",
    epsilon: float = 8/255
) -> plt.Figure:
    """
    Create side-by-side confusion matrices for clean vs adversarial.
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Get confusion matrices
    cm_clean = clean_results[seed]["confusion_matrix"]
    
    # Get adversarial CM
    if attack_key == "cw":
        cm_adv = all_results[attack_key][seed]["l2"]["confusion_matrix"]
        attack_label = "C&W L2"
    else:
        cm_adv = all_results[attack_key][seed][epsilon]["confusion_matrix"]
        eps_str = f"{int(epsilon*255)}/255"
        attack_label = f"PGD-{config.pgd_steps} (ε={eps_str})" if attack_key == "pgd" else f"FGSM (ε={eps_str})"
    
    # Normalize confusion matrices
    cm_clean_norm = cm_clean.astype(float) / cm_clean.sum(axis=1, keepdims=True)
    cm_adv_norm = cm_adv.astype(float) / cm_adv.sum(axis=1, keepdims=True)
    cm_diff = cm_adv_norm - cm_clean_norm
    
    # Plot 1: Clean CM
    im1 = axes[0].imshow(cm_clean_norm, cmap="Blues", vmin=0, vmax=1)
    axes[0].set_title(f"Clean Predictions\n(Seed {seed})", fontsize=14, fontweight="bold")
    axes[0].set_xlabel("Predicted")
    axes[0].set_ylabel("True Label")
    axes[0].set_xticks(range(len(CLASS_NAMES)))
    axes[0].set_yticks(range(len(CLASS_NAMES)))
    axes[0].set_xticklabels(CLASS_NAMES, rotation=45, ha="right")
    axes[0].set_yticklabels(CLASS_NAMES)
    
    # Add text annotations
    for i in range(len(CLASS_NAMES)):
        for j in range(len(CLASS_NAMES)):
            val = cm_clean_norm[i, j]
            color = "white" if val > 0.5 else "black"
            axes[0].text(j, i, f"{val:.2f}", ha="center", va="center", color=color, fontsize=9)
    
    plt.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04)
    
    # Plot 2: Adversarial CM
    im2 = axes[1].imshow(cm_adv_norm, cmap="Reds", vmin=0, vmax=1)
    axes[1].set_title(f"Under {attack_label}\n(Seed {seed})", fontsize=14, fontweight="bold")
    axes[1].set_xlabel("Predicted")
    axes[1].set_ylabel("True Label")
    axes[1].set_xticks(range(len(CLASS_NAMES)))
    axes[1].set_yticks(range(len(CLASS_NAMES)))
    axes[1].set_xticklabels(CLASS_NAMES, rotation=45, ha="right")
    axes[1].set_yticklabels(CLASS_NAMES)
    
    for i in range(len(CLASS_NAMES)):
        for j in range(len(CLASS_NAMES)):
            val = cm_adv_norm[i, j]
            color = "white" if val > 0.5 else "black"
            axes[1].text(j, i, f"{val:.2f}", ha="center", va="center", color=color, fontsize=9)
    
    plt.colorbar(im2, ax=axes[1], fraction=0.046, pad=0.04)
    
    # Plot 3: Difference (shows where predictions shifted)
    im3 = axes[2].imshow(cm_diff, cmap="RdBu_r", vmin=-0.5, vmax=0.5)
    axes[2].set_title(f"Prediction Shift\n(Adversarial - Clean)", fontsize=14, fontweight="bold")
    axes[2].set_xlabel("Predicted")
    axes[2].set_ylabel("True Label")
    axes[2].set_xticks(range(len(CLASS_NAMES)))
    axes[2].set_yticks(range(len(CLASS_NAMES)))
    axes[2].set_xticklabels(CLASS_NAMES, rotation=45, ha="right")
    axes[2].set_yticklabels(CLASS_NAMES)
    
    for i in range(len(CLASS_NAMES)):
        for j in range(len(CLASS_NAMES)):
            val = cm_diff[i, j]
            color = "white" if abs(val) > 0.25 else "black"
            axes[2].text(j, i, f"{val:+.2f}", ha="center", va="center", color=color, fontsize=9)
    
    cbar = plt.colorbar(im3, ax=axes[2], fraction=0.046, pad=0.04)
    cbar.set_label("Change in probability", rotation=270, labelpad=15)
    
    plt.suptitle(
        f"Confusion Matrix Analysis: Impact of {attack_label} Attack",
        fontsize=16, fontweight="bold", y=1.02
    )
    plt.tight_layout()
    
    return fig

# Create confusion matrix for first available seed
first_seed = config.seeds[0]
if first_seed in clean_results and first_seed in all_results.get("pgd", {}):
    fig_cm = create_confusion_matrix_comparison(
        clean_results, all_results, 
        seed=first_seed,
        attack_key="pgd",
        epsilon=8/255
    )
    plt.show()
    
    # Save
    if paths.figures_dir.exists():
        fig_cm.savefig(paths.figures_dir / "confusion_matrix_comparison.png", 
                       dpi=200, bbox_inches="tight", facecolor="white")
        print(f"✅ Saved to {paths.figures_dir / 'confusion_matrix_comparison.png'}")
else:
    print("⚠️ Results not available for confusion matrix visualization")

In [ ]:
#@title 🕸️ Cell 14: PhD-Level Visualization - Radar Chart & Attack Effectiveness
#@markdown **Multi-dimensional attack comparison using radar charts**

def create_radar_chart(all_results: dict, clean_results: dict, config: EvaluationConfig) -> go.Figure:
    """
    Create radar chart comparing attack effectiveness across multiple dimensions.
    """
    # Calculate mean metrics across seeds for strongest epsilon
    eps_8 = 8/255
    
    metrics = {
        "FGSM": {},
        f"PGD-{config.pgd_steps}": {},
        "C&W-L2": {}
    }
    
    dimensions = [
        "Attack Success Rate",
        "Accuracy Drop",
        "Per-Class Variance",
        "Mean Perturbation",
        "Speed (inverse)"
    ]
    
    # Calculate FGSM metrics
    if config.fgsm_enabled:
        fgsm_results = [all_results["fgsm"][s][eps_8] for s in config.seeds if s in all_results["fgsm"] and eps_8 in all_results["fgsm"][s]]
        if fgsm_results:
            metrics["FGSM"] = {
                "Attack Success Rate": np.mean([r["attack_success_rate"] for r in fgsm_results]) * 100,
                "Accuracy Drop": np.mean([r["accuracy_drop"] for r in fgsm_results]) * 100,
                "Per-Class Variance": np.mean([np.std(list(r["per_class_robust_accuracy"].values())) for r in fgsm_results]) * 100,
                "Mean Perturbation": np.mean([r["mean_linf_dist"] for r in fgsm_results]) * 255,  # Scale to 0-8
                "Speed (inverse)": 95  # FGSM is single-step, very fast
            }
    
    # Calculate PGD metrics
    if config.pgd_enabled:
        pgd_results = [all_results["pgd"][s][eps_8] for s in config.seeds if s in all_results["pgd"] and eps_8 in all_results["pgd"][s]]
        if pgd_results:
            metrics[f"PGD-{config.pgd_steps}"] = {
                "Attack Success Rate": np.mean([r["attack_success_rate"] for r in pgd_results]) * 100,
                "Accuracy Drop": np.mean([r["accuracy_drop"] for r in pgd_results]) * 100,
                "Per-Class Variance": np.mean([np.std(list(r["per_class_robust_accuracy"].values())) for r in pgd_results]) * 100,
                "Mean Perturbation": np.mean([r["mean_linf_dist"] for r in pgd_results]) * 255,
                "Speed (inverse)": 40  # PGD-40 is 40x slower than FGSM
            }
    
    # Calculate C&W metrics
    if config.cw_enabled:
        cw_results = [all_results["cw"][s]["l2"] for s in config.seeds if s in all_results["cw"] and "l2" in all_results["cw"][s]]
        if cw_results:
            metrics["C&W-L2"] = {
                "Attack Success Rate": np.mean([r["attack_success_rate"] for r in cw_results]) * 100,
                "Accuracy Drop": np.mean([r["accuracy_drop"] for r in cw_results]) * 100,
                "Per-Class Variance": np.mean([np.std(list(r["per_class_robust_accuracy"].values())) for r in cw_results]) * 100,
                "Mean Perturbation": np.mean([r["mean_l2_dist"] for r in cw_results]) * 10,  # Scale L2 for visibility
                "Speed (inverse)": 10  # C&W is slowest
            }
    
    # Create radar chart
    fig = go.Figure()
    
    colors = {"FGSM": "#FF6B6B", f"PGD-{config.pgd_steps}": "#4ECDC4", "C&W-L2": "#9B59B6"}
    
    for attack_name, data in metrics.items():
        if not data:
            continue
        
        values = [data.get(dim, 0) for dim in dimensions]
        values.append(values[0])  # Close the polygon
        
        fig.add_trace(go.Scatterpolar(
            r=values,
            theta=dimensions + [dimensions[0]],
            fill='toself',
            name=attack_name,
            line=dict(color=colors[attack_name], width=2),
            fillcolor=colors[attack_name],
            opacity=0.3
        ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100],
                ticksuffix="%"
            ),
            angularaxis=dict(
                tickfont=dict(size=12)
            )
        ),
        title=dict(
            text="<b>Attack Effectiveness Comparison (Radar Chart)</b><br>"
                 "<sup>Higher values = more effective/impactful attack</sup>",
            font=dict(size=16),
            x=0.5
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.2,
            xanchor="center",
            x=0.5
        ),
        height=600,
        width=700
    )
    
    return fig

# Create radar chart
fig_radar = create_radar_chart(all_results, clean_results, config)
fig_radar.show()

# Save
if paths.figures_dir.exists():
    fig_radar.write_html(paths.figures_dir / "attack_radar_chart.html")
    fig_radar.write_image(paths.figures_dir / "attack_radar_chart.png", scale=2)
    print(f"✅ Saved to {paths.figures_dir / 'attack_radar_chart.png'}")

In [ ]:
#@title 🖼️ Cell 15: PhD-Level Visualization - Adversarial Example Gallery
#@markdown **Visualize clean images, perturbations, and adversarial examples**

def visualize_adversarial_examples(
    all_results: dict,
    attack_key: str = "pgd",
    epsilon: float = 8/255,
    num_examples: int = 5,
    amplification: float = 10.0
) -> plt.Figure:
    """
    Create publication-quality adversarial example visualization.
    
    Shows:
    - Clean image
    - Perturbation (amplified for visibility)
    - Adversarial image
    - Prediction change
    """
    # Get saved examples from first seed
    first_seed = config.seeds[0]
    
    if attack_key == "cw":
        examples = all_results[attack_key][first_seed]["l2"].get("saved_examples", [])
    else:
        examples = all_results[attack_key][first_seed][epsilon].get("saved_examples", [])
    
    if not examples:
        print("⚠️ No saved examples available")
        return None
    
    # Select successful attacks (prediction changed)
    successful = [e for e in examples if e["clean_pred"] != e["adv_pred"]]
    if len(successful) < num_examples:
        successful = examples  # Use all if not enough successful
    
    num_examples = min(num_examples, len(successful))
    
    fig, axes = plt.subplots(num_examples, 4, figsize=(16, 4 * num_examples))
    if num_examples == 1:
        axes = axes.reshape(1, -1)
    
    for idx, example in enumerate(successful[:num_examples]):
        # Get tensors
        clean = example["clean"].numpy().transpose(1, 2, 0)  # CHW -> HWC
        adv = example["adversarial"].numpy().transpose(1, 2, 0)
        perturbation = example["perturbation"].numpy().transpose(1, 2, 0)
        
        true_label = CLASS_NAMES[example["true_label"]]
        clean_pred = CLASS_NAMES[example["clean_pred"]]
        adv_pred = CLASS_NAMES[example["adv_pred"]]
        
        # Ensure valid range
        clean = np.clip(clean, 0, 1)
        adv = np.clip(adv, 0, 1)
        
        # Amplify perturbation for visibility
        pert_amplified = perturbation * amplification + 0.5
        pert_amplified = np.clip(pert_amplified, 0, 1)
        
        # L2 and Linf norms
        l2_norm = np.sqrt(np.sum(perturbation ** 2))
        linf_norm = np.max(np.abs(perturbation))
        
        # Plot clean image
        axes[idx, 0].imshow(clean)
        axes[idx, 0].set_title(f"Clean Image\nTrue: {true_label}\nPred: {clean_pred}", fontsize=11)
        axes[idx, 0].axis("off")
        if example["clean_pred"] == example["true_label"]:
            axes[idx, 0].spines[:].set_visible(True)
            for spine in axes[idx, 0].spines.values():
                spine.set_edgecolor('green')
                spine.set_linewidth(3)
        
        # Plot perturbation
        axes[idx, 1].imshow(pert_amplified)
        axes[idx, 1].set_title(f"Perturbation (×{amplification:.0f})\nL∞: {linf_norm*255:.2f}/255\nL2: {l2_norm:.4f}", fontsize=11)
        axes[idx, 1].axis("off")
        
        # Plot adversarial image
        axes[idx, 2].imshow(adv)
        axes[idx, 2].set_title(f"Adversarial Image\nPred: {adv_pred}", fontsize=11)
        axes[idx, 2].axis("off")
        if example["adv_pred"] != example["true_label"]:
            for spine in axes[idx, 2].spines.values():
                spine.set_visible(True)
                spine.set_edgecolor('red')
                spine.set_linewidth(3)
        
        # Plot difference heatmap
        diff = np.mean(np.abs(adv - clean), axis=2)  # Average across channels
        im = axes[idx, 3].imshow(diff, cmap="hot", vmin=0, vmax=epsilon * 2)
        axes[idx, 3].set_title(f"Absolute Difference\n({clean_pred} → {adv_pred})", fontsize=11)
        axes[idx, 3].axis("off")
        plt.colorbar(im, ax=axes[idx, 3], fraction=0.046, pad=0.04)
    
    attack_label = "C&W L2" if attack_key == "cw" else f"{'PGD' if attack_key == 'pgd' else 'FGSM'}-{config.get_epsilon_str(epsilon)}"
    plt.suptitle(
        f"Adversarial Example Gallery: {attack_label} Attack\n"
        f"Green border = correct, Red border = misclassified",
        fontsize=16, fontweight="bold", y=1.02
    )
    plt.tight_layout()
    
    return fig

# Create visualization for PGD attack
if config.pgd_enabled and config.seeds[0] in all_results.get("pgd", {}):
    fig_examples = visualize_adversarial_examples(
        all_results,
        attack_key="pgd",
        epsilon=8/255,
        num_examples=5,
        amplification=10.0
    )
    if fig_examples:
        plt.show()
        
        # Save
        if paths.figures_dir.exists():
            fig_examples.savefig(paths.figures_dir / "adversarial_examples_pgd.png",
                                 dpi=200, bbox_inches="tight", facecolor="white")
            print(f"✅ Saved to {paths.figures_dir / 'adversarial_examples_pgd.png'}")

# Also show FGSM examples if available
if config.fgsm_enabled and config.seeds[0] in all_results.get("fgsm", {}):
    print("\n" + "="*60)
    print("📸 FGSM Attack Examples")
    print("="*60)
    
    fig_fgsm = visualize_adversarial_examples(
        all_results,
        attack_key="fgsm",
        epsilon=8/255,
        num_examples=3,
        amplification=10.0
    )
    if fig_fgsm:
        plt.show()
        
        if paths.figures_dir.exists():
            fig_fgsm.savefig(paths.figures_dir / "adversarial_examples_fgsm.png",
                            dpi=200, bbox_inches="tight", facecolor="white")
            print(f"✅ Saved to {paths.figures_dir / 'adversarial_examples_fgsm.png'}")

In [ ]:
#@title 📊 Cell 16: PhD-Level Analysis - Statistical Significance & Seed Consistency
#@markdown **Bootstrap confidence intervals and cross-seed consistency analysis**

from scipy import stats

def statistical_analysis(df_results: pd.DataFrame, config: EvaluationConfig) -> Dict[str, Any]:
    """
    Perform rigorous statistical analysis of adversarial evaluation results.
    """
    results = {}
    
    print("="*70)
    print("📊 STATISTICAL ANALYSIS OF ADVERSARIAL ROBUSTNESS")
    print("="*70)
    
    # 1. Cross-seed consistency (Coefficient of Variation)
    print("\n1️⃣ CROSS-SEED CONSISTENCY (Coefficient of Variation)")
    print("-" * 50)
    
    for attack in df_results["Attack"].unique():
        for eps in df_results[df_results["Attack"] == attack]["Epsilon"].unique():
            subset = df_results[(df_results["Attack"] == attack) & (df_results["Epsilon"] == eps)]
            if len(subset) >= 2:
                mean_acc = subset["Robust_Acc"].mean()
                std_acc = subset["Robust_Acc"].std()
                cv = (std_acc / mean_acc) * 100 if mean_acc > 0 else 0
                
                consistency = "✅ High" if cv < 5 else "⚠️ Medium" if cv < 10 else "❌ Low"
                print(f"   {attack:15} {eps:>7}: CV = {cv:5.2f}% {consistency}")
    
    # 2. Bootstrap 95% Confidence Intervals
    print("\n2️⃣ BOOTSTRAP 95% CONFIDENCE INTERVALS")
    print("-" * 50)
    
    def bootstrap_ci(data, n_bootstrap=1000, ci=0.95):
        """Calculate bootstrap confidence interval."""
        if len(data) < 2:
            return data.mean(), data.mean(), data.mean()
        
        bootstraps = []
        for _ in range(n_bootstrap):
            sample = np.random.choice(data, size=len(data), replace=True)
            bootstraps.append(sample.mean())
        
        lower = np.percentile(bootstraps, (1 - ci) / 2 * 100)
        upper = np.percentile(bootstraps, (1 + ci) / 2 * 100)
        return data.mean(), lower, upper
    
    ci_results = []
    for attack in df_results["Attack"].unique():
        attack_data = df_results[df_results["Attack"] == attack]
        
        if attack == "C&W-L2":
            subset = attack_data
            mean, lower, upper = bootstrap_ci(subset["Robust_Acc"].values)
            ci_results.append({
                "Attack": attack,
                "Epsilon": "N/A",
                "Mean": mean,
                "CI_Lower": lower,
                "CI_Upper": upper
            })
            print(f"   {attack:15} {'N/A':>7}: {mean:5.2f}% [{lower:5.2f}%, {upper:5.2f}%]")
        else:
            for eps in attack_data["Epsilon"].unique():
                subset = attack_data[attack_data["Epsilon"] == eps]
                mean, lower, upper = bootstrap_ci(subset["Robust_Acc"].values)
                ci_results.append({
                    "Attack": attack,
                    "Epsilon": eps,
                    "Mean": mean,
                    "CI_Lower": lower,
                    "CI_Upper": upper
                })
                print(f"   {attack:15} {eps:>7}: {mean:5.2f}% [{lower:5.2f}%, {upper:5.2f}%]")
    
    results["confidence_intervals"] = ci_results
    
    # 3. Attack Comparison (Paired t-test: FGSM vs PGD)
    print("\n3️⃣ ATTACK COMPARISON (Paired t-test at ε=8/255)")
    print("-" * 50)
    
    if config.fgsm_enabled and config.pgd_enabled:
        eps_8 = "8/255"
        fgsm_acc = df_results[(df_results["Attack"] == "FGSM") & (df_results["Epsilon"] == eps_8)]["Robust_Acc"].values
        pgd_acc = df_results[(df_results["Attack"] == f"PGD-{config.pgd_steps}") & (df_results["Epsilon"] == eps_8)]["Robust_Acc"].values
        
        if len(fgsm_acc) >= 2 and len(pgd_acc) >= 2 and len(fgsm_acc) == len(pgd_acc):
            t_stat, p_value = stats.ttest_rel(fgsm_acc, pgd_acc)
            
            significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
            
            print(f"   FGSM mean: {fgsm_acc.mean():.2f}%")
            print(f"   PGD mean:  {pgd_acc.mean():.2f}%")
            print(f"   Difference: {(fgsm_acc.mean() - pgd_acc.mean()):.2f}pp")
            print(f"   t-statistic: {t_stat:.3f}")
            print(f"   p-value: {p_value:.4f} {significance}")
            
            results["fgsm_vs_pgd"] = {
                "t_statistic": t_stat,
                "p_value": p_value,
                "significant": p_value < 0.05
            }
    
    # 4. Effect Size (Cohen's d)
    print("\n4️⃣ EFFECT SIZE (Cohen's d): Clean vs Adversarial")
    print("-" * 50)
    
    for attack in df_results["Attack"].unique():
        attack_data = df_results[df_results["Attack"] == attack]
        clean_acc = attack_data["Clean_Acc"].values
        robust_acc = attack_data["Robust_Acc"].values
        
        # Cohen's d
        pooled_std = np.sqrt((clean_acc.std()**2 + robust_acc.std()**2) / 2)
        if pooled_std > 0:
            cohens_d = (clean_acc.mean() - robust_acc.mean()) / pooled_std
        else:
            cohens_d = 0
        
        effect = "Negligible" if abs(cohens_d) < 0.2 else "Small" if abs(cohens_d) < 0.5 else "Medium" if abs(cohens_d) < 0.8 else "Large"
        print(f"   {attack:15}: d = {cohens_d:6.2f} ({effect})")
    
    print("\n" + "="*70)
    
    return results

# Run statistical analysis
stat_results = statistical_analysis(df_results, config)

# ============================================================================
# VISUALIZATION: SEED CONSISTENCY BOX PLOT
# ============================================================================

def create_seed_comparison_boxplot(df_results: pd.DataFrame) -> go.Figure:
    """Create box plot comparing robust accuracy across seeds."""
    
    fig = go.Figure()
    
    colors = {"FGSM": "#FF6B6B", f"PGD-{config.pgd_steps}": "#4ECDC4", "C&W-L2": "#9B59B6"}
    
    for attack in df_results["Attack"].unique():
        attack_data = df_results[df_results["Attack"] == attack]
        
        fig.add_trace(go.Box(
            y=attack_data["Robust_Acc"],
            x=[attack] * len(attack_data),
            name=attack,
            marker_color=colors.get(attack, "#888888"),
            boxpoints="all",
            jitter=0.3,
            pointpos=-1.8,
            hovertemplate="Seed: %{text}<br>Robust Acc: %{y:.2f}%<extra></extra>",
            text=attack_data["Seed"].astype(str)
        ))
    
    fig.update_layout(
        title=dict(
            text="<b>Cross-Seed Consistency: Robust Accuracy Distribution</b>",
            font=dict(size=16),
            x=0.5
        ),
        xaxis_title="Attack Type",
        yaxis_title="Robust Accuracy (%)",
        yaxis=dict(range=[0, 100]),
        showlegend=False,
        height=500,
        width=800
    )
    
    return fig

fig_boxplot = create_seed_comparison_boxplot(df_results)
fig_boxplot.show()

# Save
if paths.figures_dir.exists():
    fig_boxplot.write_html(paths.figures_dir / "seed_consistency_boxplot.html")
    fig_boxplot.write_image(paths.figures_dir / "seed_consistency_boxplot.png", scale=2)
    print(f"✅ Saved to {paths.figures_dir / 'seed_consistency_boxplot.png'}")

In [ ]:
#@title 💾 Cell 17: Save Results & Export for Dissertation
#@markdown **Export all results in multiple formats for dissertation use**

print("="*70)
print("💾 SAVING RESULTS & EXPORTS")
print("="*70)

# ============================================================================
# SAVE RESULTS AS CSV
# ============================================================================

# Detailed results CSV
csv_path = paths.results_dir / "adversarial_robustness_results.csv"
df_results.to_csv(csv_path, index=False)
print(f"✅ Detailed results: {csv_path}")

# Summary statistics CSV  
summary_path = paths.results_dir / "adversarial_robustness_summary.csv"
df_summary.to_csv(summary_path, index=False)
print(f"✅ Summary statistics: {summary_path}")

# ============================================================================
# SAVE AS JSON (for programmatic access)
# ============================================================================

json_results = {
    "metadata": {
        "evaluation_date": datetime.now().isoformat(),
        "seeds": config.seeds,
        "epsilons": [float(e) for e in config.epsilons],
        "attacks": {
            "fgsm": config.fgsm_enabled,
            "pgd": config.pgd_enabled,
            "pgd_steps": config.pgd_steps,
            "cw": config.cw_enabled,
            "cw_iterations": config.cw_iterations
        },
        "dataset": "ISIC 2018",
        "model": "ResNet-50",
        "num_classes": NUM_CLASSES
    },
    "clean_accuracy": {
        str(seed): {
            "accuracy": float(clean_results[seed]["accuracy"]),
            "balanced_accuracy": float(clean_results[seed]["balanced_accuracy"]),
            "f1_macro": float(clean_results[seed]["f1_macro"]),
            "auroc": float(clean_results[seed]["auroc"])
        }
        for seed in config.seeds if seed in clean_results
    },
    "adversarial_results": df_results.to_dict(orient="records")
}

json_path = paths.results_dir / "adversarial_robustness_results.json"
with open(json_path, "w") as f:
    json.dump(json_results, f, indent=2, default=str)
print(f"✅ JSON results: {json_path}")

# ============================================================================
# GENERATE LATEX TABLE FOR DISSERTATION
# ============================================================================

def generate_latex_table(df_results: pd.DataFrame, config: EvaluationConfig) -> str:
    """Generate LaTeX table for dissertation."""
    
    lines = [
        "\\begin{table}[htbp]",
        "\\centering",
        "\\caption{Adversarial Robustness Evaluation: Baseline ResNet-50 on ISIC 2018}",
        "\\label{tab:adversarial_robustness}",
        "\\begin{tabular}{llcccc}",
        "\\toprule",
        "Attack & $\\epsilon$ & Clean Acc (\\%) & Robust Acc (\\%) & Acc Drop (pp) & ASR (\\%) \\\\",
        "\\midrule"
    ]
    
    # Group and average across seeds
    for attack in df_results["Attack"].unique():
        attack_data = df_results[df_results["Attack"] == attack]
        
        for eps in attack_data["Epsilon"].unique():
            subset = attack_data[attack_data["Epsilon"] == eps]
            
            clean = f"{subset['Clean_Acc'].mean():.1f} $\\pm$ {subset['Clean_Acc'].std():.1f}"
            robust = f"{subset['Robust_Acc'].mean():.1f} $\\pm$ {subset['Robust_Acc'].std():.1f}"
            drop = f"{subset['Acc_Drop'].mean():.1f} $\\pm$ {subset['Acc_Drop'].std():.1f}"
            asr = f"{subset['Attack_Success'].mean():.1f} $\\pm$ {subset['Attack_Success'].std():.1f}"
            
            lines.append(f"{attack} & {eps} & {clean} & {robust} & {drop} & {asr} \\\\")
    
    lines.extend([
        "\\bottomrule",
        "\\end{tabular}",
        "\\begin{tablenotes}",
        "\\small",
        "\\item Note: Values are mean $\\pm$ std across seeds (42, 123, 456). ASR = Attack Success Rate.",
        "\\end{tablenotes}",
        "\\end{table}"
    ])
    
    return "\n".join(lines)

latex_table = generate_latex_table(df_results, config)

# Save LaTeX table
latex_path = paths.results_dir / "adversarial_robustness_table.tex"
with open(latex_path, "w") as f:
    f.write(latex_table)
print(f"✅ LaTeX table: {latex_path}")

# Print LaTeX table
print("\n📄 LATEX TABLE FOR DISSERTATION:")
print("-" * 60)
print(latex_table)

# ============================================================================
# LIST ALL SAVED FILES
# ============================================================================

print("\n" + "="*70)
print("📁 ALL SAVED FILES")
print("="*70)

if paths.results_dir.exists():
    for f in sorted(paths.results_dir.rglob("*")):
        if f.is_file():
            size_kb = f.stat().st_size / 1024
            print(f"   {f.relative_to(paths.results_dir)} ({size_kb:.1f} KB)")

print("="*70)

In [ ]:
#@title 📋 Cell 18: Executive Summary & Key Findings
#@markdown **Final summary of adversarial robustness evaluation**

def print_executive_summary(df_results: pd.DataFrame, clean_results: dict, config: EvaluationConfig):
    """Generate executive summary of evaluation results."""
    
    print("╔" + "═"*68 + "╗")
    print("║" + " "*20 + "EXECUTIVE SUMMARY" + " "*31 + "║")
    print("║" + " "*10 + "Phase 4: Adversarial Robustness Evaluation" + " "*15 + "║")
    print("╚" + "═"*68 + "╝")
    
    # Clean accuracy
    mean_clean = np.mean([r["accuracy"] for r in clean_results.values()]) * 100
    std_clean = np.std([r["accuracy"] for r in clean_results.values()]) * 100
    
    print(f"\n📊 BASELINE CLEAN PERFORMANCE")
    print(f"   • Clean Accuracy: {mean_clean:.2f}% ± {std_clean:.2f}%")
    print(f"   • Model: ResNet-50 (ImageNet pretrained)")
    print(f"   • Dataset: ISIC 2018 (7 classes)")
    print(f"   • Seeds evaluated: {config.seeds}")
    
    # Adversarial results summary
    print(f"\n🛡️ ADVERSARIAL ROBUSTNESS FINDINGS")
    
    # FGSM
    if config.fgsm_enabled:
        fgsm_8 = df_results[(df_results["Attack"] == "FGSM") & (df_results["Epsilon"] == "8/255")]
        if len(fgsm_8) > 0:
            mean_robust = fgsm_8["Robust_Acc"].mean()
            mean_drop = fgsm_8["Acc_Drop"].mean()
            print(f"\n   FGSM (ε=8/255):")
            print(f"   ├─ Robust Accuracy: {mean_robust:.2f}%")
            print(f"   ├─ Accuracy Drop: {mean_drop:.2f}pp")
            print(f"   └─ Interpretation: {'Moderate vulnerability' if mean_drop < 40 else 'High vulnerability'}")
    
    # PGD
    if config.pgd_enabled:
        pgd_8 = df_results[(df_results["Attack"] == f"PGD-{config.pgd_steps}") & (df_results["Epsilon"] == "8/255")]
        if len(pgd_8) > 0:
            mean_robust = pgd_8["Robust_Acc"].mean()
            mean_drop = pgd_8["Acc_Drop"].mean()
            print(f"\n   PGD-{config.pgd_steps} (ε=8/255):")
            print(f"   ├─ Robust Accuracy: {mean_robust:.2f}%")
            print(f"   ├─ Accuracy Drop: {mean_drop:.2f}pp")
            print(f"   └─ Interpretation: {'Severe vulnerability' if mean_drop > 60 else 'High vulnerability' if mean_drop > 40 else 'Moderate'}")
    
    # C&W
    if config.cw_enabled:
        cw = df_results[df_results["Attack"] == "C&W-L2"]
        if len(cw) > 0:
            mean_robust = cw["Robust_Acc"].mean()
            mean_drop = cw["Acc_Drop"].mean()
            print(f"\n   C&W L2:")
            print(f"   ├─ Robust Accuracy: {mean_robust:.2f}%")
            print(f"   ├─ Accuracy Drop: {mean_drop:.2f}pp")
            print(f"   └─ Interpretation: Strongest attack, minimal perturbation")
    
    # Key insights
    print(f"\n🔑 KEY INSIGHTS")
    print(f"   1. Standard CNNs are highly vulnerable to adversarial attacks")
    print(f"   2. PGD attacks are stronger than FGSM (multi-step > single-step)")
    print(f"   3. Some classes (e.g., MEL, BCC) may be more vulnerable than others")
    print(f"   4. Adversarial training is needed for robust medical AI deployment")
    
    # Research implications
    print(f"\n📚 RESEARCH IMPLICATIONS")
    print(f"   • These results motivate Phase 5: Adversarial Training")
    print(f"   • Tri-objective optimization will balance accuracy, robustness, and explainability")
    print(f"   • Medical AI systems require robustness validation before clinical use")
    
    # Files generated
    print(f"\n📁 GENERATED OUTPUTS")
    print(f"   • Results CSV: adversarial_robustness_results.csv")
    print(f"   • Summary CSV: adversarial_robustness_summary.csv")
    print(f"   • JSON results: adversarial_robustness_results.json")
    print(f"   • LaTeX table: adversarial_robustness_table.tex")
    print(f"   • Figures: robustness_curves.png, vulnerability_heatmap.png, etc.")
    
    print("\n" + "═"*70)
    print(f"✅ Phase 4 Adversarial Robustness Evaluation COMPLETE")
    print(f"⏱️  Completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("═"*70)

# Print executive summary
print_executive_summary(df_results, clean_results, config)

# ============================================================================
# FINAL MEMORY CLEANUP
# ============================================================================

gc.collect()
torch.cuda.empty_cache()
print("\n🧹 Memory cleaned up")